In [ ]:
!pip install transformers tensorboardX

     |████████████████████████████████| 124 kB 7.3 MB/s 


In [ ]:
## 1 Data Preparation
## 1.1 Base

import os

if not os.path.isdir('/content/data/'):
    !mkdir /content/data/
!cp /content/drive/MyDrive/RoBERTa_base_data/combined_train.txt /content/data
!cp /content/drive/MyDrive/RoBERTa_base_data/combined_dev.txt /content/data
!mv /content/data/combined_train.txt /content/data/train.txt
!mv /content/data/combined_dev.txt /content/data/dev.txt

!split -n 15 /content/data/train.txt

!mv /content/xaa /content/data/train00.txt
!mv /content/xab /content/data/train01.txt
!mv /content/xac /content/data/train02.txt
!mv /content/xad /content/data/train03.txt
!mv /content/xae /content/data/train04.txt
!mv /content/xaf /content/data/train05.txt
!mv /content/xag /content/data/train06.txt
!mv /content/xah /content/data/train07.txt
!mv /content/xai /content/data/train08.txt
!mv /content/xaj /content/data/train09.txt
!mv /content/xak /content/data/train10.txt
!mv /content/xal /content/data/train11.txt
!mv /content/xam /content/data/train12.txt
!mv /content/xan /content/data/train13.txt
!mv /content/xao /content/data/train14.txt


!split -n 15 /content/data/dev.txt

!mv /content/xaa /content/data/dev00.txt
!mv /content/xab /content/data/dev01.txt
!mv /content/xac /content/data/dev02.txt
!mv /content/xad /content/data/dev03.txt
!mv /content/xae /content/data/dev04.txt
!mv /content/xaf /content/data/dev05.txt
!mv /content/xag /content/data/dev06.txt
!mv /content/xah /content/data/dev07.txt
!mv /content/xai /content/data/dev08.txt
!mv /content/xaj /content/data/dev09.txt
!mv /content/xak /content/data/dev10.txt
!mv /content/xal /content/data/dev11.txt
!mv /content/xam /content/data/dev12.txt
!mv /content/xan /content/data/dev13.txt
!mv /content/xao /content/data/dev14.txt


In [ ]:
## 1.2 Domain

!mkdir -p /content/data/biomed
!mkdir -p /content/data/cs
!mkdir -p /content/data/review
!mkdir -p /content/data/news

!wc -l /content/drive/MyDrive/RoBERTa_domain_data/domain_news.txt    # 4561839
!wc -l /content/drive/MyDrive/RoBERTa_domain_data/domain_review.txt  # 35265731
!wc -l /content/drive/MyDrive/RoBERTa_domain_data/domain_cs.txt      # 712923
!wc -l /content/drive/MyDrive/RoBERTa_domain_data/domain_biomed.txt  # 905793

TRAIN_SIZE = int(712923 * 0.8)
VAL_SIZE = 712923 - TRAIN_SIZE

4561839 /content/drive/MyDrive/RoBERTa_domain_data/domain_news.txt
35265731 /content/drive/MyDrive/RoBERTa_domain_data/domain_review.txt
712923 /content/drive/MyDrive/RoBERTa_domain_data/domain_cs.txt
905793 /content/drive/MyDrive/RoBERTa_domain_data/domain_biomed.txt


In [ ]:
TRAIN_SIZE =  570338#@param {type:"integer"}
VAL_SIZE =  142585#@param {type: "integer"}

In [ ]:
## Every domain specific corpus has 570338 lines
!(head -n $TRAIN_SIZE /content/drive/MyDrive/RoBERTa_domain_data/domain_cs.txt) > /content/data/cs/train.txt
!(sed -n {TRAIN_SIZE + 1},{TRAIN_SIZE + VAL_SIZE}p /content/drive/MyDrive/RoBERTa_domain_data/domain_cs.txt) > /content/data/cs/dev.txt

!(head -n $TRAIN_SIZE /content/drive/MyDrive/RoBERTa_domain_data/domain_biomed.txt) > /content/data/biomed/train.txt
!(sed -n {TRAIN_SIZE + 1},{TRAIN_SIZE + VAL_SIZE}p /content/drive/MyDrive/RoBERTa_domain_data/domain_biomed.txt) > /content/data/biomed/dev.txt

!(head -n $TRAIN_SIZE /content/drive/MyDrive/RoBERTa_domain_data/domain_review.txt) > /content/data/review/train.txt
!(sed -n {TRAIN_SIZE + 1},{TRAIN_SIZE + VAL_SIZE}p /content/drive/MyDrive/RoBERTa_domain_data/domain_review.txt) > /content/data/review/dev.txt

!(head -n $TRAIN_SIZE /content/drive/MyDrive/RoBERTa_domain_data/domain_news.txt) > /content/data/news/train.txt
!(sed -n {TRAIN_SIZE + 1},{TRAIN_SIZE + VAL_SIZE}p /content/drive/MyDrive/RoBERTa_domain_data/domain_news.txt) > /content/data/news/dev.txt

In [ ]:
## 2 Train tokenizer
from tokenizers import ByteLevelBPETokenizer

corpus = '/content/drive/MyDrive/RoBERTa_base_data/combined_corpus.txt'
tokenizer = ByteLevelBPETokenizer
tokenizer.train(files=corpus,
                vocab_size=50265,
                min_frequency=2,
                special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])
if not os.path.isdir('/content/models/roberta/'):
    !mkdir -p "/content/models/roberta"
tokenizer.save('/content/models/roberta/tokenizer.json', pretty=True)

In [ ]:
## 2.1 Copy Trained Tokenizer
import os
if not os.path.isdir('/content/models/roberta'):
    !mkdir -p "/content/models/roberta"
!cp /content/drive/MyDrive/Adaptive_pretrain/tokenizer.json /content/models/roberta/

In [ ]:
## 3 run_language_model.py
%%writefile run_lm.py

### THIS FILE IS COPIED FROM THE HUGGINGFACE REPOSITORY FOR CONVENIENCE.

# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""


import argparse
import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import numpy as np
import torch

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.auto import tqdm, trange

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter


logger = logging.getLogger(__name__)


MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)


class TextDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, file_path: str, block_size=512):
        assert os.path.isfile(file_path)

        block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size) + "_" + filename
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            with open(file_path, encoding="utf-8") as f:
                text = f.read()

            tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

            for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                self.examples.append(tokenizer.build_inputs_with_special_tokens(tokenized_text[i : i + block_size]))
            # Note that we are loosing the last truncated example here for the sake of simplicity (no padding)
            # If your dataset is small, first you should loook for a bigger one :-) and second you
            # can change this behavior by adding (model specific) padding.

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)


class LineByLineTextDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, file_path: str, block_size=512):
        assert os.path.isfile(file_path)
        # Here, we do not cache the features, operating under the assumption
        # that we will soon use fast multithreaded tokenizers from the
        # `tokenizers` repo everywhere =)
        logger.info("Creating features from dataset file at %s", file_path)

        with open(file_path, encoding="utf-8") as f:
            lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

        self.examples = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=block_size)["input_ids"]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return torch.tensor(self.examples[i], dtype=torch.long)


def load_and_cache_examples(args, tokenizer, evaluate=False):
    file_path = args.eval_data_file if evaluate else args.train_data_file
    if args.line_by_line:
        return LineByLineTextDataset(tokenizer, args, file_path=file_path, block_size=args.block_size)
    else:
        return TextDataset(tokenizer, args, file_path=file_path, block_size=args.block_size)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)


def mask_tokens(inputs: torch.Tensor, tokenizer: PreTrainedTokenizer, args) -> Tuple[torch.Tensor, torch.Tensor]:
    """ Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original. """

    if tokenizer.mask_token is None:
        raise ValueError(
            "This tokenizer does not have a mask token which is necessary for masked language modeling. Remove the --mlm flag if you want to use this tokenizer."
        )

    labels = inputs.clone()
    # We sample a few tokens in each sequence for masked-LM training (with probability args.mlm_probability defaults to 0.15 in Bert/RoBERTa)
    probability_matrix = torch.full(labels.shape, args.mlm_probability)
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
    ]
    probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
    if tokenizer._pad_token is not None:
        padding_mask = labels.eq(tokenizer.pad_token_id)
        probability_matrix.masked_fill_(padding_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # We only compute loss on masked tokens

    # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    # 10% of the time, we replace masked input tokens with random word
    indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
    random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
    inputs[indices_random] = random_words[indices_random]

    # The rest of the time (10% of the time) we keep the masked input tokens unchanged
    return inputs, labels


def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        # try:
        #     # set global_step to gobal_step of last saved checkpoint from model path
        #     checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
        #     global_step = int(checkpoint_suffix)
        #     epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
        #     steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)
            # steps_trained_in_current_epoch = 0

        #     logger.info("  Continuing training from checkpoint, will skip to saved global_step")
        #     logger.info("  Continuing training from epoch %d", epochs_trained)
        #     logger.info("  Continuing training from global step %d", global_step)
        #     logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        # except ValueError:
        #     logger.info("  Starting fine-tuning.")
        checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
        global_step = int(checkpoint_suffix)
        epochs_trained = 0

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for epoch in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])

        if args.local_rank != -1:
            train_sampler.set_epoch(epoch)

        for step, batch in enumerate(epoch_iterator):
            
            # Skip past any already trained steps if resuming training
            # if steps_trained_in_current_epoch > 0:
            #     steps_trained_in_current_epoch -= 1
            #     continue

            inputs, labels = mask_tokens(batch, tokenizer, args) if args.mlm else (batch, batch)
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            # outputs = model(inputs, masked_lm_labels=labels) if args.mlm else model(inputs, labels=labels)
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                # GPU:
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step


def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, evaluate=True)

    if args.local_rank in [-1, 0]:
        os.makedirs(eval_output_dir, exist_ok=True)

    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = mask_tokens(batch, tokenizer, args) if args.mlm else (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            # outputs = model(inputs, masked_lm_labels=labels) if args.mlm else model(inputs, labels=labels)
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))
    print("Eval loss = " + str(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result


def main():
    parser = argparse.ArgumentParser()

    # Required parameters
    parser.add_argument(
        "--train_data_file", default=None, type=str, required=True, help="The input training data file (a text file)."
    )
    parser.add_argument(
        "--output_dir",
        type=str,
        required=True,
        help="The output directory where the model predictions and checkpoints will be written.",
    )
    parser.add_argument(
        "--model_type", type=str, required=True, help="The model architecture to be trained or fine-tuned.",
    )

    # Other parameters
    parser.add_argument(
        "--eval_data_file",
        default=None,
        type=str,
        help="An optional input evaluation data file to evaluate the perplexity on (a text file).",
    )
    parser.add_argument(
        "--line_by_line",
        action="store_true",
        help="Whether distinct lines of text in the dataset are to be handled as distinct sequences.",
    )
    parser.add_argument(
        "--should_continue", action="store_true", help="Whether to continue from latest checkpoint in output_dir"
    )
    parser.add_argument(
        "--model_name_or_path",
        default=None,
        type=str,
        help="The model checkpoint for weights initialization. Leave None if you want to train a model from scratch.",
    )

    parser.add_argument(
        "--mlm", action="store_true", help="Train with masked-language modeling loss instead of language modeling."
    )
    parser.add_argument(
        "--mlm_probability", type=float, default=0.15, help="Ratio of tokens to mask for masked language modeling loss"
    )

    parser.add_argument(
        "--config_name",
        default=None,
        type=str,
        help="Optional pretrained config name or path if not the same as model_name_or_path. If both are None, initialize a new config.",
    )
    parser.add_argument(
        "--tokenizer_name",
        default=None,
        type=str,
        help="Optional pretrained tokenizer name or path if not the same as model_name_or_path. If both are None, initialize a new tokenizer.",
    )
    parser.add_argument(
        "--cache_dir",
        default=None,
        type=str,
        help="Optional directory to store the pre-trained models downloaded from s3 (instead of the default one)",
    )
    parser.add_argument(
        "--block_size",
        default=-1,
        type=int,
        help="Optional input sequence length after tokenization."
        "The training dataset will be truncated in block of this size for training."
        "Default to the model max input length for single sentence inputs (take into account special tokens).",
    )
    parser.add_argument("--do_train", action="store_true", help="Whether to run training.")
    parser.add_argument("--do_eval", action="store_true", help="Whether to run eval on the dev set.")
    parser.add_argument(
        "--evaluate_during_training", action="store_true", help="Run evaluation during training at each logging step."
    )

    parser.add_argument("--per_gpu_train_batch_size", default=4, type=int, help="Batch size per GPU/CPU for training.")
    parser.add_argument(
        "--per_gpu_eval_batch_size", default=4, type=int, help="Batch size per GPU/CPU for evaluation."
    )
    parser.add_argument(
        "--gradient_accumulation_steps",
        type=int,
        default=1,
        help="Number of updates steps to accumulate before performing a backward/update pass.",
    )
    parser.add_argument("--learning_rate", default=5e-5, type=float, help="The initial learning rate for Adam.")
    parser.add_argument("--weight_decay", default=0.0, type=float, help="Weight decay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
    parser.add_argument(
        "--num_train_epochs", default=1.0, type=float, help="Total number of training epochs to perform."
    )
    parser.add_argument(
        "--max_steps",
        default=-1,
        type=int,
        help="If > 0: set total number of training steps to perform. Override num_train_epochs.",
    )
    parser.add_argument("--warmup_steps", default=0, type=int, help="Linear warmup over warmup_steps.")

    parser.add_argument("--logging_steps", type=int, default=500, help="Log every X updates steps.")
    parser.add_argument("--save_steps", type=int, default=500, help="Save checkpoint every X updates steps.")
    parser.add_argument(
        "--save_total_limit",
        type=int,
        default=None,
        help="Limit the total amount of checkpoints, delete the older checkpoints in the output_dir, does not delete by default",
    )
    parser.add_argument(
        "--eval_all_checkpoints",
        action="store_true",
        help="Evaluate all checkpoints starting with the same prefix as model_name_or_path ending and ending with step number",
    )
    parser.add_argument("--no_cuda", action="store_true", help="Avoid using CUDA when available")
    parser.add_argument(
        "--overwrite_output_dir", action="store_true", help="Overwrite the content of the output directory"
    )
    parser.add_argument(
        "--overwrite_cache", action="store_true", help="Overwrite the cached training and evaluation sets"
    )
    parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")

    parser.add_argument(
        "--fp16",
        action="store_true",
        help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit",
    )
    parser.add_argument(
        "--fp16_opt_level",
        type=str,
        default="O1",
        help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
        "See details at https://nvidia.github.io/apex/amp.html",
    )
    parser.add_argument("--local_rank", type=int, default=-1, help="For distributed training: local_rank")
    parser.add_argument("--server_ip", type=str, default="", help="For distant debugging.")
    parser.add_argument("--server_port", type=str, default="", help="For distant debugging.")
    args = parser.parse_args()

    if args.model_type in ["bert", "roberta", "distilbert", "camembert"] and not args.mlm:
        raise ValueError(
            "BERT and RoBERTa-like models do not have LM heads but masked LM heads. They must be run using the --mlm "
            "flag (masked language modeling)."
        )
    if args.eval_data_file is None and args.do_eval:
        raise ValueError(
            "Cannot do evaluation without an evaluation data file. Either supply a file to --eval_data_file "
            "or remove the --do_eval argument."
        )
    # if args.should_continue:
    sorted_checkpoints = _sorted_checkpoints(args)
    if len(sorted_checkpoints) == 0:
        raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
    else:
        args.model_name_or_path = sorted_checkpoints[-1]
        print("Load checkpoint from " + args.model_name_or_path)

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup distant debugging if needed
    if args.server_ip and args.server_port:
        # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
        import ptvsd

        print("Waiting for debugger attach")
        ptvsd.enable_attach(address=(args.server_ip, args.server_port), redirect_output=True)
        ptvsd.wait_for_attach()

    # Setup CUDA, GPU & distributed training
    if args.local_rank == -1 or args.no_cuda:
        # try: ## Setup tpu
        #     device = xm.xla_device()
        #     args.n_gpu = 1
        # except:
        #     device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        #     args.n_gpu = 0 if args.no_cuda else torch.cuda.device_count()
        device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        args.n_gpu = 0 if args.no_cuda else torch.cuda.device_count()
    else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.cuda.set_device(args.local_rank)
        device = torch.device("cuda", args.local_rank)
        torch.distributed.init_process_group(backend="nccl")
        args.n_gpu = 1
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    # Load pretrained model and tokenizer
    if args.local_rank not in [-1, 0]:
        torch.distributed.barrier()  # Barrier to make sure only the first process in distributed training download model & vocab

    if args.config_name:
        config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    elif args.model_name_or_path:
        config = AutoConfig.from_pretrained(args.model_name_or_path, cache_dir=args.cache_dir)
    else:
        # When we release a pip version exposing CONFIG_MAPPING,
        # we can do `config = CONFIG_MAPPING[args.model_type]()`.
        raise ValueError(
            "You are instantiating a new config instance from scratch. This is not supported, but you can do it from another script, save it,"
            "and load it from here, using --config_name"
        )

    if args.tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    elif args.model_name_or_path:
        tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=args.cache_dir)
    else:
        raise ValueError(
            "You are instantiating a new tokenizer from scratch. This is not supported, but you can do it from another script, save it,"
            "and load it from here, using --tokenizer_name"
        )

    if args.block_size <= 0:
        # args.block_size = tokenizer.max_len
        args.block_size = 512
        # Our input block size will be the max possible for the model
    else:
        # args.block_size = min(args.block_size, tokenizer.max_len)
        args.block_size = min(args.block_size, 512)

    if args.model_name_or_path:
        model = AutoModelWithLMHead.from_pretrained(
            args.model_name_or_path,
            from_tf=bool(".ckpt" in args.model_name_or_path),
            config=config,
            cache_dir=args.cache_dir,
        )
    else:
        logger.info("Training new model from scratch")
        model = AutoModelWithLMHead.from_config(config)

    model.to(args.device)

    if args.local_rank == 0:
        torch.distributed.barrier()  # End of barrier to make sure only the first process in distributed training download model & vocab

    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        if args.local_rank not in [-1, 0]:
            torch.distributed.barrier()  # Barrier to make sure only the first process in distributed training process the dataset, and the others will use the cache

        train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False)

        if args.local_rank == 0:
            torch.distributed.barrier()

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
        # Create output directory if needed
        if args.local_rank in [-1, 0]:
            os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results


if __name__ == "__main__":
    main()

Overwriting run_lm.py


In [ ]:
## 4 Config
import json

config = {
    "architectures": [
		"RobertaForMaskedLM"
	],
	"attention_probs_dropout_prob": 0.1,
	"hidden_act": "gelu",
	"hidden_dropout_prob": 0.1,
	"hidden_size": 768,
	"initializer_range": 0.02,
	"intermediate_size": 3072,
	"layer_norm_eps": 1e-05,
	"max_position_embeddings": 514,
	"model_type": "roberta",
	"num_attention_heads": 12,
	"num_hidden_layers": 12,
	"type_vocab_size": 1,
	"vocab_size": 50265
}

tokenizer_config = {"max_len": 512}

with open("/content/models/roberta/config.json", 'w') as fp:
    json.dump(config, fp)

with open("/content/models/roberta/tokenizer_config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)

In [ ]:
## 5 Model Parameters
MODEL_TYPE = "roberta" #@param ["roberta", "bert"]
MODEL_DIR = "/content/models/roberta" #@param {type: "string"}
OUTPUT_DIR = "/content/models/roberta/output_base" #@param {type: "string"}
TRAIN_PATH = "/content/data/train.txt" #@param {type: "string"}
EVAL_PATH = "/content/data/dev.txt" #@param {type: "string"}

In [ ]:
## 6 Base Command
cmd = """python run_lm.py \
    --output_dir {output_dir} \
    --model_type {model_type} \
    --mlm \
    --config_name {config_name} \
    --tokenizer_name {tokenizer_name} \
    {line_by_line} \
    {should_continue} \
    {model_name_or_path} \
    --train_data_file {train_path} \
    --eval_data_file {eval_path} \
    --num_train_epochs {num_train_epochs} \
    --do_train \
    {do_eval} \
    {evaluate_during_training} \
    --overwrite_output_dir \
    --block_size 512 \
    --warmup_steps 10 \
    --learning_rate 5e-5 \
    --per_gpu_train_batch_size 8 \
    --per_gpu_eval_batch_size 8 \
    --gradient_accumulation_steps 4 \
    --weight_decay 0.01 \
    --adam_epsilon 1e-6 \
    --max_grad_norm 100.0 \
    --save_total_limit 10 \
    --save_steps 100 \
    --logging_steps 5 \
    --seed 42 \
    |& tee -a /content/drive/MyDrive/Adaptive_pretrain/base_output_5k.txt
"""

In [ ]:
## 7 Training Parameters
train_params = {
    "output_dir": OUTPUT_DIR,
    "model_type": MODEL_TYPE,
    "config_name": MODEL_DIR,
    "tokenizer_name": MODEL_DIR,
    "train_path": TRAIN_PATH,
    "eval_path": EVAL_PATH,
    "num_train_epochs": 1.0,
    "do_eval": "--do_eval",
    "evaluate_during_training": "",
    "line_by_line": "--line_by_line",
    "should_continue": "",
    "model_name_or_path": "",
}

In [ ]:
## Setup TPU
import tensorflow as tf

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print(tf.config.list_logical_devices("TPU"))

import os
assert os.environ['COLAB_TPU_ADDR']

!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

import torch
import torch_xla
import torch_xla.core.xla_model as xm

INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.94.29.114:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.94.29.114:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


[LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]
  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl (149.9 MB)


In [ ]:
# Copy last trained model to MODEL_DIR
!cp -R /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base5/models /content/
!cp -R /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base5/runs /content/

In [ ]:
!cp /content/drive/MyDrive/Adaptive_pretrain/tokenizer.json /content/models/roberta/

In [ ]:
!rm -r /content/models/roberta/output_base/checkpoint-163600
!rm -r /content/models/roberta/output_base/checkpoint-163700
!rm -r /content/models/roberta/output_base/checkpoint-163800
!rm -r /content/models/roberta/output_base/checkpoint-163900
!rm -r /content/models/roberta/output_base/checkpoint-164000
!rm -r /content/models/roberta/output_base/checkpoint-164100
!rm -r /content/models/roberta/output_base/checkpoint-164200
!rm -r /content/models/roberta/output_base/checkpoint-164300
!rm -r /content/models/roberta/output_base/checkpoint-164400
!rm -r /content/models/roberta/output_base/checkpoint-164500
!rm -r /content/models/roberta/output_base/checkpoint-164600
!rm -r /content/models/roberta/output_base/checkpoint-164700
!rm -r /content/models/roberta/output_base/checkpoint-164800
!rm -r /content/models/roberta/output_base/checkpoint-164900
!rm -r /content/models/roberta/output_base/checkpoint-165000
!rm -r /content/models/roberta/output_base/checkpoint-165100
!rm -r /content/models/roberta/output_base/checkpoint-165200
!rm -r /content/models/roberta/output_base/checkpoint-165300
!rm -r /content/models/roberta/output_base/checkpoint-165400
!rm -r /content/models/roberta/output_base/checkpoint-165500
!rm -r /content/models/roberta/output_base/checkpoint-165600
!rm -r /content/models/roberta/output_base/checkpoint-165700
!rm -r /content/models/roberta/output_base/checkpoint-165800
!rm -r /content/models/roberta/output_base/checkpoint-165900
!rm -r /content/models/roberta/output_base/checkpoint-166000
!rm -r /content/models/roberta/output_base/checkpoint-166100
!rm -r /content/models/roberta/output_base/checkpoint-166200
!rm -r /content/models/roberta/output_base/checkpoint-166300
!rm -r /content/models/roberta/output_base/checkpoint-166400
!rm -r /content/models/roberta/output_base/checkpoint-166500
!rm -r /content/models/roberta/output_base/checkpoint-166600
!rm -r /content/models/roberta/output_base/checkpoint-166700
!rm -r /content/models/roberta/output_base/checkpoint-166800
!rm -r /content/models/roberta/output_base/checkpoint-166900
!rm -r /content/models/roberta/output_base/checkpoint-167000
!rm -r /content/models/roberta/output_base/checkpoint-167100
!rm -r /content/models/roberta/output_base/checkpoint-167200
!rm -r /content/models/roberta/output_base/checkpoint-167300
!rm -r /content/models/roberta/output_base/checkpoint-167400

In [ ]:
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-163600
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-163700
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-163800
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-163900
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-164000
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-164100
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-164200
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-164300
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-164400
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-164500
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-164600
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-164700
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-164800
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-164900
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-165000
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-165100
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-165200
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-165300
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-165400
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-165500
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-165600
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-165700
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-165800
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-165900
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-166000
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-166100
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-166200
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-166300
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-166400
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-166500
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-166600
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-166700
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-166800
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-166900
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-167000
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-167100
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-167200
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-167300
!rm -r /content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base1/models/roberta/output_base/checkpoint-167400


In [ ]:
## 8.1 Train RoBERTa base

for num_corpus in range(6, 15):
    if num_corpus < 10:
        print("__________START " + f"train0{num_corpus}__________")
        train_file = f"train0{num_corpus}.txt"
        dev_file = f"dev0{num_corpus}.txt"
    else:
        print("__________START " + f"train{num_corpus}__________")
        train_file = f"train{num_corpus}.txt"
        dev_file = f"dev{num_corpus}.txt"
    
    # Model Parameters
    MODEL_TYPE = "roberta"
    MODEL_DIR = "/content/models/roberta"
    OUTPUT_DIR = "/content/models/roberta/output_base"
    TRAIN_PATH = "/content/drive/MyDrive/Adaptive_pretrain/data/"+train_file
    EVAL_PATH = "/content/drive/MyDrive/Adaptive_pretrain/data/"+dev_file

    train_params = {
        "output_dir": OUTPUT_DIR,
        "model_type": MODEL_TYPE,
        "config_name": MODEL_DIR,
        "tokenizer_name": MODEL_DIR,
        "train_path": TRAIN_PATH,
        "eval_path": EVAL_PATH,
        "num_train_epochs": 1.0,
        "do_eval": "--do_eval",
        "evaluate_during_training": "",
        "line_by_line": "--line_by_line",
        "should_continue": "",
        "model_name_or_path": "",
    }

    # if num_corpus != 0:
    #     train_params['should_continue'] = "--should_continue"
    
    !{cmd.format(**train_params)}

    save_dir = f'/content/drive/MyDrive/Adaptive_pretrain/roberta_models_5k_base{num_corpus}/'
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    !cp -R /content/models "$save_dir"
    !cp -R /content/runs "$save_dir"

Streaming output truncated to the last 5000 lines.
Iteration:   5%|▌         | 15598/306948 [23:43<5:53:03, 13.75it/s]09/17/2021 05:56:49 - INFO - __main__ -   Saving model checkpoint to /content/models/roberta/output_base/checkpoint-615000
09/17/2021 05:56:49 - INFO - __main__ -   Deleting older checkpoint [/content/models/roberta/output_base/checkpoint-614000] due to args.save_total_limit
09/17/2021 05:56:53 - INFO - __main__ -   Saving optimizer and scheduler states to /content/models/roberta/output_base/checkpoint-615000

Iteration:   5%|▌         | 15998/306948 [24:20<6:05:06, 13.28it/s]09/17/2021 05:57:26 - INFO - __main__ -   Saving model checkpoint to /content/models/roberta/output_base/checkpoint-615100
09/17/2021 05:57:26 - INFO - __main__ -   Deleting older checkpoint [/content/models/roberta/output_base/checkpoint-614100] due to args.save_total_limit
09/17/2021 05:57:30 - INFO - __main__ -   Saving optimizer and scheduler states to /content/models/roberta/output_base/checkp

In [ ]:
## 8.2 Domain Command
cmd = """python run_lm.py \
    --output_dir {output_dir} \
    --model_type {model_type} \
    --mlm \
    --config_name {config_name} \
    --tokenizer_name {tokenizer_name} \
    {line_by_line} \
    {should_continue} \
    {model_name_or_path} \
    --train_data_file {train_path} \
    --eval_data_file {eval_path} \
    --num_train_epochs {num_train_epochs} \
    --do_train \
    {do_eval} \
    {evaluate_during_training} \
    --overwrite_output_dir \
    --block_size 512 \
    --max_step 2500 \
    --warmup_steps 10 \
    --learning_rate 5e-5 \
    --per_gpu_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --weight_decay 0.01 \
    --adam_epsilon 1e-6 \
    --max_grad_norm 100.0 \
    --save_total_limit 50 \
    --save_steps 100 \
    --logging_steps 5 \
    --seed 42 \
    |& tee -a /content/drive/MyDrive/Adaptive_pretrain/domain_output_2k.txt
"""

In [ ]:
## 8.3 Train RoBERTa Domain

domains = ['cs', 'biomed', 'review', 'news']
base = '/content/data/'
for d in domains:
    print("__________START " + f"{d}__________")
    train_file = f"{d}/train.txt"
    dev_file = f"{d}/dev.txt"
    
    # COPY the latest model
    !cp -R /content/drive/MyDrive/Adaptive_pretrain/sentenced/roberta_models_base9/models/ /content/
    !cp -R /content/drive/MyDrive/Adaptive_pretrain/sentenced/roberta_models_base9/runs/ /content/
    !cp /content/drive/MyDrive/Adaptive_pretrain/tokenizer.json /content/models/roberta/

    with open("/content/models/roberta/config.json", 'w') as fp:
        json.dump(config, fp)

    with open("/content/models/roberta/tokenizer_config.json", 'w') as fp:
        json.dump(tokenizer_config, fp)
    
    # Model Parameters
    MODEL_TYPE = "roberta"
    MODEL_DIR = "/content/models/roberta"
    OUTPUT_DIR = "/content/models/roberta/output_base"
    TRAIN_PATH = "/content/data/"+train_file
    EVAL_PATH = "/content/data/"+dev_file

    train_params = {
        "output_dir": OUTPUT_DIR,
        "model_type": MODEL_TYPE,
        "config_name": MODEL_DIR,
        "tokenizer_name": MODEL_DIR,
        "train_path": TRAIN_PATH,
        "eval_path": EVAL_PATH,
        "num_train_epochs": 1.0,
        "do_eval": "--do_eval",
        "evaluate_during_training": "",
        "line_by_line": "--line_by_line",
        "should_continue": "--should_continue",
        "model_name_or_path": "",
    }

    !{cmd.format(**train_params)}

    save_dir = f'/content/drive/MyDrive/Adaptive_pretrain/roberta_models_domain_2k_{d}/'
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    !cp -R /content/models "$save_dir"
    !cp -R /content/runs "$save_dir"

__________START cs__________
^C
^C
__________START biomed__________
^C
09/07/2021 04:23:54 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:592: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Load checkpoint from /content/models/roberta/output_base/checkpoint-168300
Traceback (most recent call last):
  File "run_lm.py", line 806, in <module>
    main()
  File "run_lm.py", line 733, in main
    cache_dir=args.cache_dir,
  File "/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py", line 594, in from_pretrained
    return super().from_pretrained(pretrained_model_name_or_path,

In [ ]:
## 9 Cross domain
## 9.1 cs -> biomed

cmd = """python run_lm.py \
    --output_dir {output_dir} \
    --model_type {model_type} \
    --mlm \
    --config_name {config_name} \
    --tokenizer_name {tokenizer_name} \
    {line_by_line} \
    {should_continue} \
    {model_name_or_path} \
    --train_data_file {train_path} \
    --eval_data_file {eval_path} \
    --num_train_epochs {num_train_epochs} \
    --do_train \
    {do_eval} \
    {evaluate_during_training} \
    --overwrite_output_dir \
    --block_size 512 \
    --max_step 2500 \
    --warmup_steps 10 \
    --learning_rate 5e-5 \
    --per_gpu_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --weight_decay 0.01 \
    --adam_epsilon 1e-6 \
    --max_grad_norm 100.0 \
    --save_total_limit 50 \
    --save_steps 100 \
    --logging_steps 5 \
    --seed 42 \
    |& tee -a /content/drive/MyDrive/Adaptive_pretrain/cs2biomed_output_2k.txt
"""

train_file = "biomed/train.txt"
dev_file = "biomed/dev.txt"

!cp -R /content/drive/MyDrive/Adaptive_pretrain/roberta_models_domain/roberta_models_domain_cs/models/ /content/
!cp -R /content/drive/MyDrive/Adaptive_pretrain/roberta_models_domain/roberta_models_domain_cs/runs/ /content/

!cp /content/drive/MyDrive/Adaptive_pretrain/tokenizer.json /content/models/roberta/

with open("/content/models/roberta/config.json", 'w') as fp:
    json.dump(config, fp)

with open("/content/models/roberta/tokenizer_config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)

# Model Parameters
MODEL_TYPE = "roberta"
MODEL_DIR = "/content/models/roberta"
OUTPUT_DIR = "/content/models/roberta/output_base"
TRAIN_PATH = "/content/data/"+train_file
EVAL_PATH = "/content/data/"+dev_file

train_params = {
    "output_dir": OUTPUT_DIR,
    "model_type": MODEL_TYPE,
    "config_name": MODEL_DIR,
    "tokenizer_name": MODEL_DIR,
    "train_path": TRAIN_PATH,
    "eval_path": EVAL_PATH,
    "num_train_epochs": 1.0,
    "do_eval": "--do_eval",
    "evaluate_during_training": "",
    "line_by_line": "--line_by_line",
    "should_continue": "--should_continue",
    "model_name_or_path": "",
}

!{cmd.format(**train_params)}

save_dir = f'/content/drive/MyDrive/Adaptive_pretrain/roberta_models_domain_cs2biomed_2k/'
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)
!cp -R /content/models "$save_dir"
!cp -R /content/runs "$save_dir"

09/03/2021 11:13:42 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:592: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
09/03/2021 11:13:50 - INFO - __main__ -   Training/evaluation parameters Namespace(adam_epsilon=1e-06, block_size=512, cache_dir=None, config_name='/content/models/roberta', device=device(type='cuda'), do_eval=True, do_train=True, eval_all_checkpoints=False, eval_data_file='/content/data/biomed/dev.txt', evaluate_during_training=False, fp16=False, fp16_opt_level='O1', gradient_accumulation_steps=4, learning_rate=5e-05, line_by_line=True, local_rank=-1, logging_steps=2, max_grad_norm

In [ ]:
## 9.2 biomed -> news

cmd = """python run_lm.py \
    --output_dir {output_dir} \
    --model_type {model_type} \
    --mlm \
    --config_name {config_name} \
    --tokenizer_name {tokenizer_name} \
    {line_by_line} \
    {should_continue} \
    {model_name_or_path} \
    --train_data_file {train_path} \
    --eval_data_file {eval_path} \
    --num_train_epochs {num_train_epochs} \
    --do_train \
    {do_eval} \
    {evaluate_during_training} \
    --overwrite_output_dir \
    --block_size 512 \
    --max_step 2500 \
    --warmup_steps 10 \
    --learning_rate 5e-5 \
    --per_gpu_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --weight_decay 0.01 \
    --adam_epsilon 1e-6 \
    --max_grad_norm 100.0 \
    --save_total_limit 50 \
    --save_steps 100 \
    --logging_steps 5 \
    --seed 42 \
    |& tee -a /content/drive/MyDrive/Adaptive_pretrain/biomed2news_output_2k.txt
"""

train_file = "news/train.txt"
dev_file = "news/dev.txt"

!cp -R /content/drive/MyDrive/Adaptive_pretrain/roberta_models_domain/roberta_models_domain_biomed/models/ /content/
!cp -R /content/drive/MyDrive/Adaptive_pretrain/roberta_models_domain/roberta_models_domain_biomed/runs/ /content/

!cp /content/drive/MyDrive/Adaptive_pretrain/tokenizer.json /content/models/roberta/

with open("/content/models/roberta/config.json", 'w') as fp:
    json.dump(config, fp)

with open("/content/models/roberta/tokenizer_config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)

# Model Parameters
MODEL_TYPE = "roberta"
MODEL_DIR = "/content/models/roberta"
OUTPUT_DIR = "/content/models/roberta/output_base"
TRAIN_PATH = "/content/data/"+train_file
EVAL_PATH = "/content/data/"+dev_file

train_params = {
    "output_dir": OUTPUT_DIR,
    "model_type": MODEL_TYPE,
    "config_name": MODEL_DIR,
    "tokenizer_name": MODEL_DIR,
    "train_path": TRAIN_PATH,
    "eval_path": EVAL_PATH,
    "num_train_epochs": 1.0,
    "do_eval": "--do_eval",
    "evaluate_during_training": "",
    "line_by_line": "--line_by_line",
    "should_continue": "--should_continue",
    "model_name_or_path": "",
}

!{cmd.format(**train_params)}

save_dir = f'/content/drive/MyDrive/Adaptive_pretrain/roberta_models_domain_biomed2news_2k/'
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)
!cp -R /content/models "$save_dir"
!cp -R /content/runs "$save_dir"

09/03/2021 11:52:47 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:592: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
09/03/2021 11:52:56 - INFO - __main__ -   Training/evaluation parameters Namespace(adam_epsilon=1e-06, block_size=512, cache_dir=None, config_name='/content/models/roberta', device=device(type='cuda'), do_eval=True, do_train=True, eval_all_checkpoints=False, eval_data_file='/content/data/news/dev.txt', evaluate_during_training=False, fp16=False, fp16_opt_level='O1', gradient_accumulation_steps=4, learning_rate=5e-05, line_by_line=True, local_rank=-1, logging_steps=2, max_grad_norm=1

In [ ]:
## 9.3 biomed -> review

cmd = """python run_lm.py \
    --output_dir {output_dir} \
    --model_type {model_type} \
    --mlm \
    --config_name {config_name} \
    --tokenizer_name {tokenizer_name} \
    {line_by_line} \
    {should_continue} \
    {model_name_or_path} \
    --train_data_file {train_path} \
    --eval_data_file {eval_path} \
    --num_train_epochs {num_train_epochs} \
    --do_train \
    {do_eval} \
    {evaluate_during_training} \
    --overwrite_output_dir \
    --block_size 512 \
    --max_step 125000 \
    --warmup_steps 10 \
    --learning_rate 5e-5 \
    --per_gpu_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --weight_decay 0.01 \
    --adam_epsilon 1e-6 \
    --max_grad_norm 100.0 \
    --save_total_limit 10 \
    --save_steps 10 \
    --logging_steps 2 \
    --seed 42 \
    |& tee -a /content/drive/MyDrive/Adaptive_pretrain/biomed2review_output.txt
"""

train_file = "review/train.txt"
dev_file = "review/dev.txt"

!cp -R /content/drive/MyDrive/Adaptive_pretrain/roberta_models_domain/roberta_models_domain_biomed/models/ /content/
!cp -R /content/drive/MyDrive/Adaptive_pretrain/roberta_models_domain/roberta_models_domain_biomed/runs/ /content/

!cp /content/drive/MyDrive/Adaptive_pretrain/tokenizer.json /content/models/roberta/

with open("/content/models/roberta/config.json", 'w') as fp:
    json.dump(config, fp)

with open("/content/models/roberta/tokenizer_config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)

# Model Parameters
MODEL_TYPE = "roberta"
MODEL_DIR = "/content/models/roberta"
OUTPUT_DIR = "/content/models/roberta/output_base"
TRAIN_PATH = "/content/data/"+train_file
EVAL_PATH = "/content/data/"+dev_file

train_params = {
    "output_dir": OUTPUT_DIR,
    "model_type": MODEL_TYPE,
    "config_name": MODEL_DIR,
    "tokenizer_name": MODEL_DIR,
    "train_path": TRAIN_PATH,
    "eval_path": EVAL_PATH,
    "num_train_epochs": 1.0,
    "do_eval": "--do_eval",
    "evaluate_during_training": "--evaluate_during_training",
    "line_by_line": "--line_by_line",
    "should_continue": "--should_continue",
    "model_name_or_path": "",
}

!{cmd.format(**train_params)}

save_dir = f'/content/drive/MyDrive/Adaptive_pretrain/roberta_models_domain_biomed2review/'
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)
!cp -R /content/models "$save_dir"
!cp -R /content/runs "$save_dir"

Streaming output truncated to the last 5000 lines.
Evaluating:  53%|█████▎    | 18992/35522 [05:34<04:31, 60.95it/s]

Evaluating:  53%|█████▎    | 18999/35522 [05:34<04:37, 59.51it/s]

Evaluating:  54%|█████▎    | 19006/35522 [05:34<04:25, 62.12it/s]

Evaluating:  54%|█████▎    | 19013/35522 [05:34<04:27, 61.69it/s]

Evaluating:  54%|█████▎    | 19020/35522 [05:35<04:19, 63.61it/s]

Evaluating:  54%|█████▎    | 19027/35522 [05:35<04:40, 58.84it/s]

Evaluating:  54%|█████▎    | 19033/35522 [05:35<04:40, 58.71it/s]

Evaluating:  54%|█████▎    | 19039/35522 [05:35<05:19, 51.67it/s]

Evaluating:  54%|█████▎    | 19046/35522 [05:35<04:55, 55.70it/s]

Evaluating:  54%|█████▎    | 19052/35522 [05:35<04:59, 55.05it/s]

Evaluating:  54%|█████▎    | 19059/35522 [05:35<04:43, 58.11it/s]

Evaluating:  54%|█████▎    | 19066/35522 [05:35<04:42, 58.29it/s]

Evaluating:  54%|█████▎    | 19073/35522 [05:35<04:33, 60.17it/s]

Evaluating:  54%|█████▎    | 19080/35522 [05:36<04:26, 61.72it/s]

Evaluating:

In [ ]:
!rm -r /content/models/roberta/output_base
!rm -r /content/runs/

In [ ]:
!split -n 10 /content/data/train.txt

In [ ]:
!mv /content/xaa /content/data/train00.txt
!mv /content/xab /content/data/train01.txt
!mv /content/xac /content/data/train02.txt
!mv /content/xad /content/data/train03.txt
!mv /content/xae /content/data/train04.txt
!mv /content/xaf /content/data/train05.txt
!mv /content/xag /content/data/train06.txt
!mv /content/xah /content/data/train07.txt
!mv /content/xai /content/data/train08.txt
!mv /content/xaj /content/data/train09.txt

In [ ]:
!split -n 10 /content/data/dev.txt

In [ ]:
!mv /content/xaa /content/data/dev00.txt
!mv /content/xab /content/data/dev01.txt
!mv /content/xac /content/data/dev02.txt
!mv /content/xad /content/data/dev03.txt
!mv /content/xae /content/data/dev04.txt
!mv /content/xaf /content/data/dev05.txt
!mv /content/xag /content/data/dev06.txt
!mv /content/xah /content/data/dev07.txt
!mv /content/xai /content/data/dev08.txt
!mv /content/xaj /content/data/dev09.txt

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 7.0 MB/s 
     |████████████████████████████████| 50 kB 8.4 MB/s 
     |████████████████████████████████| 3.3 MB 85.2 MB/s 
     |████████████████████████████████| 895 kB 39.1 MB/s 
     |████████████████████████████████| 636 kB 87.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%%writefile run_lm.py

### THIS FILE IS COPIED FROM THE HUGGINGFACE REPOSITORY FOR CONVENIENCE.

# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""


import argparse
import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter


logger = logging.getLogger(__name__)


MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)


class TextDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, file_path: str, block_size=512):
        assert os.path.isfile(file_path)

        block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size) + "_" + filename
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            with open(file_path, encoding="utf-8") as f:
                text = f.read()

            tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

            for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                self.examples.append(tokenizer.build_inputs_with_special_tokens(tokenized_text[i : i + block_size]))
            # Note that we are loosing the last truncated example here for the sake of simplicity (no padding)
            # If your dataset is small, first you should loook for a bigger one :-) and second you
            # can change this behavior by adding (model specific) padding.

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)


class LineByLineTextDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, file_path: str, block_size=512):
        assert os.path.isfile(file_path)
        # Here, we do not cache the features, operating under the assumption
        # that we will soon use fast multithreaded tokenizers from the
        # `tokenizers` repo everywhere =)
        logger.info("Creating features from dataset file at %s", file_path)

        with open(file_path, encoding="utf-8") as f:
            lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

        self.examples = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=block_size)["input_ids"]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return torch.tensor(self.examples[i], dtype=torch.long)


def load_and_cache_examples(args, tokenizer, evaluate=False):
    file_path = args.eval_data_file if evaluate else args.train_data_file
    if args.line_by_line:
        return LineByLineTextDataset(tokenizer, args, file_path=file_path, block_size=args.block_size)
    else:
        return TextDataset(tokenizer, args, file_path=file_path, block_size=args.block_size)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)


def mask_tokens(inputs: torch.Tensor, tokenizer: PreTrainedTokenizer, args) -> Tuple[torch.Tensor, torch.Tensor]:
    """ Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original. """

    if tokenizer.mask_token is None:
        raise ValueError(
            "This tokenizer does not have a mask token which is necessary for masked language modeling. Remove the --mlm flag if you want to use this tokenizer."
        )

    labels = inputs.clone()
    # We sample a few tokens in each sequence for masked-LM training (with probability args.mlm_probability defaults to 0.15 in Bert/RoBERTa)
    probability_matrix = torch.full(labels.shape, args.mlm_probability)
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
    ]
    probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
    if tokenizer._pad_token is not None:
        padding_mask = labels.eq(tokenizer.pad_token_id)
        probability_matrix.masked_fill_(padding_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # We only compute loss on masked tokens

    # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    # 10% of the time, we replace masked input tokens with random word
    indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
    random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
    inputs[indices_random] = random_words[indices_random]

    # The rest of the time (10% of the time) we keep the masked input tokens unchanged
    return inputs, labels


def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for epoch in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])

        if args.local_rank != -1:
            train_sampler.set_epoch(epoch)

        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = mask_tokens(batch, tokenizer, args) if args.mlm else (batch, batch)
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            # outputs = model(inputs, masked_lm_labels=labels) if args.mlm else model(inputs, labels=labels)
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step


def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, evaluate=True)

    if args.local_rank in [-1, 0]:
        os.makedirs(eval_output_dir, exist_ok=True)

    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating", position=0, leave=True):
        inputs, labels = mask_tokens(batch, tokenizer, args) if args.mlm else (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            # outputs = model(inputs, masked_lm_labels=labels) if args.mlm else model(inputs, labels=labels)
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result


def main():
    parser = argparse.ArgumentParser()

    # Required parameters
    parser.add_argument(
        "--train_data_file", default=None, type=str, required=True, help="The input training data file (a text file)."
    )
    parser.add_argument(
        "--output_dir",
        type=str,
        required=True,
        help="The output directory where the model predictions and checkpoints will be written.",
    )
    parser.add_argument(
        "--model_type", type=str, required=True, help="The model architecture to be trained or fine-tuned.",
    )

    # Other parameters
    parser.add_argument(
        "--eval_data_file",
        default=None,
        type=str,
        help="An optional input evaluation data file to evaluate the perplexity on (a text file).",
    )
    parser.add_argument(
        "--line_by_line",
        action="store_true",
        help="Whether distinct lines of text in the dataset are to be handled as distinct sequences.",
    )
    parser.add_argument(
        "--should_continue", action="store_true", help="Whether to continue from latest checkpoint in output_dir"
    )
    parser.add_argument(
        "--model_name_or_path",
        default=None,
        type=str,
        help="The model checkpoint for weights initialization. Leave None if you want to train a model from scratch.",
    )

    parser.add_argument(
        "--mlm", action="store_true", help="Train with masked-language modeling loss instead of language modeling."
    )
    parser.add_argument(
        "--mlm_probability", type=float, default=0.15, help="Ratio of tokens to mask for masked language modeling loss"
    )

    parser.add_argument(
        "--config_name",
        default=None,
        type=str,
        help="Optional pretrained config name or path if not the same as model_name_or_path. If both are None, initialize a new config.",
    )
    parser.add_argument(
        "--tokenizer_name",
        default=None,
        type=str,
        help="Optional pretrained tokenizer name or path if not the same as model_name_or_path. If both are None, initialize a new tokenizer.",
    )
    parser.add_argument(
        "--cache_dir",
        default=None,
        type=str,
        help="Optional directory to store the pre-trained models downloaded from s3 (instead of the default one)",
    )
    parser.add_argument(
        "--block_size",
        default=-1,
        type=int,
        help="Optional input sequence length after tokenization."
        "The training dataset will be truncated in block of this size for training."
        "Default to the model max input length for single sentence inputs (take into account special tokens).",
    )
    parser.add_argument("--do_train", action="store_true", help="Whether to run training.")
    parser.add_argument("--do_eval", action="store_true", help="Whether to run eval on the dev set.")
    parser.add_argument(
        "--evaluate_during_training", action="store_true", help="Run evaluation during training at each logging step."
    )

    parser.add_argument("--per_gpu_train_batch_size", default=4, type=int, help="Batch size per GPU/CPU for training.")
    parser.add_argument(
        "--per_gpu_eval_batch_size", default=4, type=int, help="Batch size per GPU/CPU for evaluation."
    )
    parser.add_argument(
        "--gradient_accumulation_steps",
        type=int,
        default=1,
        help="Number of updates steps to accumulate before performing a backward/update pass.",
    )
    parser.add_argument("--learning_rate", default=5e-5, type=float, help="The initial learning rate for Adam.")
    parser.add_argument("--weight_decay", default=0.0, type=float, help="Weight decay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
    parser.add_argument(
        "--num_train_epochs", default=1.0, type=float, help="Total number of training epochs to perform."
    )
    parser.add_argument(
        "--max_steps",
        default=-1,
        type=int,
        help="If > 0: set total number of training steps to perform. Override num_train_epochs.",
    )
    parser.add_argument("--warmup_steps", default=0, type=int, help="Linear warmup over warmup_steps.")

    parser.add_argument("--logging_steps", type=int, default=500, help="Log every X updates steps.")
    parser.add_argument("--save_steps", type=int, default=500, help="Save checkpoint every X updates steps.")
    parser.add_argument(
        "--save_total_limit",
        type=int,
        default=None,
        help="Limit the total amount of checkpoints, delete the older checkpoints in the output_dir, does not delete by default",
    )
    parser.add_argument(
        "--eval_all_checkpoints",
        action="store_true",
        help="Evaluate all checkpoints starting with the same prefix as model_name_or_path ending and ending with step number",
    )
    parser.add_argument("--no_cuda", action="store_true", help="Avoid using CUDA when available")
    parser.add_argument(
        "--overwrite_output_dir", action="store_true", help="Overwrite the content of the output directory"
    )
    parser.add_argument(
        "--overwrite_cache", action="store_true", help="Overwrite the cached training and evaluation sets"
    )
    parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")

    parser.add_argument(
        "--fp16",
        action="store_true",
        help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit",
    )
    parser.add_argument(
        "--fp16_opt_level",
        type=str,
        default="O1",
        help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
        "See details at https://nvidia.github.io/apex/amp.html",
    )
    parser.add_argument("--local_rank", type=int, default=-1, help="For distributed training: local_rank")
    parser.add_argument("--server_ip", type=str, default="", help="For distant debugging.")
    parser.add_argument("--server_port", type=str, default="", help="For distant debugging.")
    args = parser.parse_args()

    if args.model_type in ["bert", "roberta", "distilbert", "camembert"] and not args.mlm:
        raise ValueError(
            "BERT and RoBERTa-like models do not have LM heads but masked LM heads. They must be run using the --mlm "
            "flag (masked language modeling)."
        )
    if args.eval_data_file is None and args.do_eval:
        raise ValueError(
            "Cannot do evaluation without an evaluation data file. Either supply a file to --eval_data_file "
            "or remove the --do_eval argument."
        )
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup distant debugging if needed
    if args.server_ip and args.server_port:
        # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
        import ptvsd

        print("Waiting for debugger attach")
        ptvsd.enable_attach(address=(args.server_ip, args.server_port), redirect_output=True)
        ptvsd.wait_for_attach()

    # Setup CUDA, GPU & distributed training
    if args.local_rank == -1 or args.no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        args.n_gpu = 0 if args.no_cuda else torch.cuda.device_count()
    else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.cuda.set_device(args.local_rank)
        device = torch.device("cuda", args.local_rank)
        torch.distributed.init_process_group(backend="nccl")
        args.n_gpu = 1
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    # Load pretrained model and tokenizer
    if args.local_rank not in [-1, 0]:
        torch.distributed.barrier()  # Barrier to make sure only the first process in distributed training download model & vocab

    if args.config_name:
        config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    elif args.model_name_or_path:
        config = AutoConfig.from_pretrained(args.model_name_or_path, cache_dir=args.cache_dir)
    else:
        # When we release a pip version exposing CONFIG_MAPPING,
        # we can do `config = CONFIG_MAPPING[args.model_type]()`.
        raise ValueError(
            "You are instantiating a new config instance from scratch. This is not supported, but you can do it from another script, save it,"
            "and load it from here, using --config_name"
        )

    if args.tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    elif args.model_name_or_path:
        tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=args.cache_dir)
    else:
        raise ValueError(
            "You are instantiating a new tokenizer from scratch. This is not supported, but you can do it from another script, save it,"
            "and load it from here, using --tokenizer_name"
        )

    if args.block_size <= 0:
        # args.block_size = tokenizer.max_len
        args.block_size = 512
        # Our input block size will be the max possible for the model
    else:
        # args.block_size = min(args.block_size, tokenizer.max_len)
        args.block_size = min(args.block_size, 512)

    if args.model_name_or_path:
        model = AutoModelWithLMHead.from_pretrained(
            args.model_name_or_path,
            from_tf=bool(".ckpt" in args.model_name_or_path),
            config=config,
            cache_dir=args.cache_dir,
        )
    else:
        logger.info("Training new model from scratch")
        model = AutoModelWithLMHead.from_config(config)

    model.to(args.device)

    if args.local_rank == 0:
        torch.distributed.barrier()  # End of barrier to make sure only the first process in distributed training download model & vocab

    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        if args.local_rank not in [-1, 0]:
            torch.distributed.barrier()  # Barrier to make sure only the first process in distributed training process the dataset, and the others will use the cache

        train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False)

        if args.local_rank == 0:
            torch.distributed.barrier()

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
        # Create output directory if needed
        if args.local_rank in [-1, 0]:
            os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results


if __name__ == "__main__":
    main()

Writing run_lm.py


In [ ]:
import os
from tokenizers import ByteLevelBPETokenizer as bpe

corpus = '/content/drive/MyDrive/RoBERTa_base_data/combined_corpus.txt'
tokenizer = bpe()
tokenizer.train(files=corpus,
                vocab_size=50265,
                min_frequency=2,
                special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])
if not os.path.isdir('/content/models/roberta/'):
    !mkdir -p "/content/models/roberta"
tokenizer.save('/content/models/roberta/tokenizer.json', pretty=True)

In [ ]:
!mkdir -p "/content/models/roberta"
!cp /content/drive/MyDrive/Adaptive_pretrain/tokenizer.json /content/models/roberta/

In [ ]:
import tensorflow as tf
import torch

In [ ]:
print(tf.config.list_physical_devices("GPU"))
print(torch.cuda.is_available())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True


In [ ]:
from tokenizers import Tokenizer

test_tokenizer = Tokenizer.from_file('/content/models/roberta/tokenizer.json')
encoded = test_tokenizer.encode("This is a test!!")
print(encoded.tokens)

['This', 'Ġis', 'Ġa', 'Ġtest', '!!']


In [ ]:
import json

config = {
    "architectures": [
		"RobertaForMaskedLM"
	],
	"attention_probs_dropout_prob": 0.1,
	"hidden_act": "gelu",
	"hidden_dropout_prob": 0.1,
	"hidden_size": 768,
	"initializer_range": 0.02,
	"intermediate_size": 3072,
	"layer_norm_eps": 1e-05,
	"max_position_embeddings": 514,
	"model_type": "roberta",
	"num_attention_heads": 12,
	"num_hidden_layers": 12,
	"type_vocab_size": 1,
	"vocab_size": 50265
}

with open("/content/models/roberta/config.json", 'w') as fp:
    json.dump(config, fp)

tokenizer_config = {"max_len": 512}

with open("/content/models/roberta/tokenizer_config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)

In [ ]:
import os

if not os.path.isdir('/content/data/'):
    !mkdir /content/data/
!cp /content/drive/MyDrive/RoBERTa_base_data/combined_train.txt /content/data
!cp /content/drive/MyDrive/RoBERTa_base_data/combined_dev.txt /content/data
!mv /content/data/combined_train.txt /content/data/train.txt
!mv /content/data/combined_dev.txt /content/data/dev.txt

In [ ]:
# Model paths
MODEL_TYPE = "roberta" #@param ["roberta", "bert"]
MODEL_DIR = "/content/models/roberta" #@param {type: "string"}
OUTPUT_DIR = "/content/models/roberta/output_base" #@param {type: "string"}
TRAIN_PATH = "/content/data/train01.txt" #@param {type: "string"}
EVAL_PATH = "/content/data/dev01.txt" #@param {type: "string"}

In [ ]:
# Command line
cmd = """python run_lm.py \
    --output_dir {output_dir} \
    --model_type {model_type} \
    --mlm \
    --config_name {config_name} \
    --tokenizer_name {tokenizer_name} \
    {line_by_line} \
    {should_continue} \
    {model_name_or_path} \
    --train_data_file {train_path} \
    --eval_data_file {eval_path} \
    --num_train_epochs {num_train_epochs} \
    --do_train \
    {do_eval} \
    {evaluate_during_training} \
    --overwrite_output_dir \
    --block_size 512 \
    --max_step 25 \
    --warmup_steps 10 \
    --learning_rate 5e-5 \
    --per_gpu_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --weight_decay 0.01 \
    --adam_epsilon 1e-6 \
    --max_grad_norm 100.0 \
    --save_total_limit 10 \
    --save_steps 10 \
    --logging_steps 2 \
    --seed 42 \
    |& tee -a /content/drive/MyDrive/Adaptive_pretrain/domain_output.txt
"""

In [ ]:
# Arguments for training from scratch. I turn off evaluate_during_training,
#   line_by_line, should_continue, and model_name_or_path.
train_params = {
    "output_dir": OUTPUT_DIR,
    "model_type": MODEL_TYPE,
    "config_name": MODEL_DIR,
    "tokenizer_name": MODEL_DIR,
    "train_path": TRAIN_PATH,
    "eval_path": EVAL_PATH,
    "num_train_epochs": 3.0,
    "do_eval": "--do_eval",
    "evaluate_during_training": "",
    "line_by_line": "--line_by_line",
    "should_continue": "",
    "model_name_or_path": "",
}

In [ ]:
!{cmd.format(**train_params)}

08/31/2021 03:39:53 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/31/2021 03:39:53 - INFO - __main__ -   Training new model from scratch
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:892: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
08/31/2021 03:40:03 - INFO - __main__ -   Training/evaluation parameters Namespace(adam_epsilon=1e-06, block_size=512, cache_dir=None, config_name='/content/models/roberta', device=device(type='cuda'), do_eval=True, do_train=True, eval_all_checkpoints=False, eval_data_file='/content/data/dev01.txt', evaluate_during_training=False, fp16=False, fp16_opt_level='O1', gradient_accumulation_steps=4, learning_rate=5e

In [ ]:
## Transfer Trained model back to drive
if not os.path.isdir('/content/drive/MyDrive/Adaptive_pretrain/roberta_models_base/'):
    !mkdir '/content/drive/MyDrive/Adaptive_pretrain/roberta_models_base'
!cp -R /content/models /content/drive/MyDrive/Adaptive_pretrain/roberta_models_base/
!cp -R /content/runs /content/drive/MyDrive/Adaptive_pretrain/roberta_models_base/

In [ ]:
## Check
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model='/content/models/roberta/output_base',
    tokenizer='/content/models/roberta/output_base'
)

In [ ]:
# Train RoBERTa base

import os
# from distutils.dir_util import copy_tree
for num_corpus in range(0, 10):
    print("__________START " + f"train0{num_corpus}__________")
    train_file = f"train0{num_corpus}.txt"
    dev_file = f"dev0{num_corpus}.txt"
    # Model paths
    MODEL_TYPE = "roberta"
    MODEL_DIR = "/content/models/roberta"
    OUTPUT_DIR = "/content/models/roberta/output_base"
    TRAIN_PATH = "/content/data/"+train_file
    EVAL_PATH = "/content/data/"+dev_file

    train_params = {
        "output_dir": OUTPUT_DIR,
        "model_type": MODEL_TYPE,
        "config_name": MODEL_DIR,
        "tokenizer_name": MODEL_DIR,
        "train_path": TRAIN_PATH,
        "eval_path": EVAL_PATH,
        "num_train_epochs": 1.0,
        "do_eval": "--do_eval",
        "evaluate_during_training": "",
        "line_by_line": "--line_by_line",
        "should_continue": "",
        "model_name_or_path": "",
    }

    if num_corpus != 0:
        train_params['should_continue'] = "--should_continue"
    
    !{cmd.format(**train_params)}

    save_dir = f'/content/drive/MyDrive/Adaptive_pretrain/roberta_models_base_3ep{num_corpus}/'
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    !cp -R /content/models "$save_dir"
    !cp -R /content/runs "$save_dir"

__________START train00__________
09/03/2021 01:41:35 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
09/03/2021 01:41:36 - INFO - __main__ -   Training new model from scratch
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:582: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
09/03/2021 01:41:45 - INFO - __main__ -   Training/evaluation parameters Namespace(adam_epsilon=1e-06, block_size=512, cache_dir=None, config_name='/content/models/roberta', device=device(type='cuda'), do_eval=True, do_train=True, eval_all_checkpoints=False, eval_data_file='/content/data/dev00.txt', evaluate_during_training=False, fp16=False, fp16_opt_level='O1', gradient_accu

In [ ]:
output_dir = f'/content/drive/MyDrive/Adaptive_pretrain/roberta_models_base{num_corpus}/'
!cp -R /content/models "$output_dir"

In [ ]:
!echo "START" |& tee /content/drive/MyDrive/Adaptive_pretrain/base_output.txt

START


In [ ]:
from torch.utils.data import Dataset, DataLoader, RandomSampler
import os
import logging

from transformers import (
    AdamW,
    PreTrainedModel,
    PreTrainedTokenizer
)

In [ ]:
class LineByLineDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, file_path: str, block_size=512):
        assert os.path.isfile(file_path)
        logger.info("Creating features from dataset file at %s.", file_path)

        with open(file_path, encoding="utf-8") as f:
            lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]
        self.examples = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=block_size)['input_ids']
    
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, i):
        return torch.tensor(self.example[i], dtype=torch.long)

In [ ]:
!rm -r /content/models
!rm -r /content/runs

In [ ]:
!mkdir -p "/content/models/roberta"

In [ ]:
import os
from tokenizers import ByteLevelBPETokenizer as bpe

corpus = '/content/drive/MyDrive/RoBERTa_base_data/combined_train.txt'
tokenizer = bpe()
tokenizer.train(files=corpus,
                vocab_size=51200,
                min_frequency=2,
                special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])
if not os.path.isdir('/content/models/roberta/'):
    !mkdir -p "/content/models/roberta"
tokenizer.save('/content/models/roberta/tokenizer.json', pretty=True)

In [ ]:
from tokenizers import Tokenizer

test_tokenizer = Tokenizer.from_file('/content/models/roberta/tokenizer.json')
encoded = test_tokenizer.encode("This is a test")
print(encoded.tokens)

['This', 'Ġis', 'Ġa', 'Ġtest']


In [ ]:
from typing import Dict, List, Tuple
from torch.nn.utils.rnn import pad_sequence

In [ ]:
def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

In [ ]:
import json

config = {
    "architectures": [
		"RobertaForMaskedLM"
	],
	"attention_probs_dropout_prob": 0.1,
	"hidden_act": "gelu",
	"hidden_dropout_prob": 0.1,
	"hidden_size": 768,
	"initializer_range": 0.02,
	"intermediate_size": 3072,
	"layer_norm_eps": 1e-05,
	"max_position_embeddings": 514,
	"model_type": "roberta",
	"num_attention_heads": 12,
	"num_hidden_layers": 12,
	"type_vocab_size": 1,
	"vocab_size": 51200
}

with open("/content/models/roberta/config.json", 'w') as fp:
    json.dump(config, fp)

tokenizer_config = {"max_len": 512}

with open("/content/models/roberta/tokenizer_config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)

In [ ]:
%%writefile run_lm.py

### THIS FILE IS COPIED FROM THE HUGGINGFACE REPOSITORY FOR CONVENIENCE.

# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""


import argparse
import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter


logger = logging.getLogger(__name__)


MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)


class TextDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, file_path: str, block_size=512):
        assert os.path.isfile(file_path)

        block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size) + "_" + filename
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            with open(file_path, encoding="utf-8") as f:
                text = f.read()

            tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

            for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                self.examples.append(tokenizer.build_inputs_with_special_tokens(tokenized_text[i : i + block_size]))
            # Note that we are loosing the last truncated example here for the sake of simplicity (no padding)
            # If your dataset is small, first you should loook for a bigger one :-) and second you
            # can change this behavior by adding (model specific) padding.

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)


class LineByLineTextDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, file_path: str, block_size=512):
        assert os.path.isfile(file_path)
        # Here, we do not cache the features, operating under the assumption
        # that we will soon use fast multithreaded tokenizers from the
        # `tokenizers` repo everywhere =)
        logger.info("Creating features from dataset file at %s", file_path)

        with open(file_path, encoding="utf-8") as f:
            lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

        self.examples = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=block_size)["input_ids"]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return torch.tensor(self.examples[i], dtype=torch.long)


def load_and_cache_examples(args, tokenizer, evaluate=False):
    file_path = args.eval_data_file if evaluate else args.train_data_file
    if args.line_by_line:
        return LineByLineTextDataset(tokenizer, args, file_path=file_path, block_size=args.block_size)
    else:
        return TextDataset(tokenizer, args, file_path=file_path, block_size=args.block_size)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)


def mask_tokens(inputs: torch.Tensor, tokenizer: PreTrainedTokenizer, args) -> Tuple[torch.Tensor, torch.Tensor]:
    """ Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original. """

    if tokenizer.mask_token is None:
        raise ValueError(
            "This tokenizer does not have a mask token which is necessary for masked language modeling. Remove the --mlm flag if you want to use this tokenizer."
        )

    labels = inputs.clone()
    # We sample a few tokens in each sequence for masked-LM training (with probability args.mlm_probability defaults to 0.15 in Bert/RoBERTa)
    probability_matrix = torch.full(labels.shape, args.mlm_probability)
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
    ]
    probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
    if tokenizer._pad_token is not None:
        padding_mask = labels.eq(tokenizer.pad_token_id)
        probability_matrix.masked_fill_(padding_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # We only compute loss on masked tokens

    # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    # 10% of the time, we replace masked input tokens with random word
    indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
    random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
    inputs[indices_random] = random_words[indices_random]

    # The rest of the time (10% of the time) we keep the masked input tokens unchanged
    return inputs, labels


def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for epoch in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])

        if args.local_rank != -1:
            train_sampler.set_epoch(epoch)

        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = mask_tokens(batch, tokenizer, args) if args.mlm else (batch, batch)
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            # outputs = model(inputs, masked_lm_labels=labels) if args.mlm else model(inputs, labels=labels)
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step


def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, evaluate=True)

    if args.local_rank in [-1, 0]:
        os.makedirs(eval_output_dir, exist_ok=True)

    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = mask_tokens(batch, tokenizer, args) if args.mlm else (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            # outputs = model(inputs, masked_lm_labels=labels) if args.mlm else model(inputs, labels=labels)
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result


def main():
    parser = argparse.ArgumentParser()

    # Required parameters
    parser.add_argument(
        "--train_data_file", default=None, type=str, required=True, help="The input training data file (a text file)."
    )
    parser.add_argument(
        "--output_dir",
        type=str,
        required=True,
        help="The output directory where the model predictions and checkpoints will be written.",
    )
    parser.add_argument(
        "--model_type", type=str, required=True, help="The model architecture to be trained or fine-tuned.",
    )

    # Other parameters
    parser.add_argument(
        "--eval_data_file",
        default=None,
        type=str,
        help="An optional input evaluation data file to evaluate the perplexity on (a text file).",
    )
    parser.add_argument(
        "--line_by_line",
        action="store_true",
        help="Whether distinct lines of text in the dataset are to be handled as distinct sequences.",
    )
    parser.add_argument(
        "--should_continue", action="store_true", help="Whether to continue from latest checkpoint in output_dir"
    )
    parser.add_argument(
        "--model_name_or_path",
        default=None,
        type=str,
        help="The model checkpoint for weights initialization. Leave None if you want to train a model from scratch.",
    )

    parser.add_argument(
        "--mlm", action="store_true", help="Train with masked-language modeling loss instead of language modeling."
    )
    parser.add_argument(
        "--mlm_probability", type=float, default=0.15, help="Ratio of tokens to mask for masked language modeling loss"
    )

    parser.add_argument(
        "--config_name",
        default=None,
        type=str,
        help="Optional pretrained config name or path if not the same as model_name_or_path. If both are None, initialize a new config.",
    )
    parser.add_argument(
        "--tokenizer_name",
        default=None,
        type=str,
        help="Optional pretrained tokenizer name or path if not the same as model_name_or_path. If both are None, initialize a new tokenizer.",
    )
    parser.add_argument(
        "--cache_dir",
        default=None,
        type=str,
        help="Optional directory to store the pre-trained models downloaded from s3 (instead of the default one)",
    )
    parser.add_argument(
        "--block_size",
        default=-1,
        type=int,
        help="Optional input sequence length after tokenization."
        "The training dataset will be truncated in block of this size for training."
        "Default to the model max input length for single sentence inputs (take into account special tokens).",
    )
    parser.add_argument("--do_train", action="store_true", help="Whether to run training.")
    parser.add_argument("--do_eval", action="store_true", help="Whether to run eval on the dev set.")
    parser.add_argument(
        "--evaluate_during_training", action="store_true", help="Run evaluation during training at each logging step."
    )

    parser.add_argument("--per_gpu_train_batch_size", default=4, type=int, help="Batch size per GPU/CPU for training.")
    parser.add_argument(
        "--per_gpu_eval_batch_size", default=4, type=int, help="Batch size per GPU/CPU for evaluation."
    )
    parser.add_argument(
        "--gradient_accumulation_steps",
        type=int,
        default=1,
        help="Number of updates steps to accumulate before performing a backward/update pass.",
    )
    parser.add_argument("--learning_rate", default=5e-5, type=float, help="The initial learning rate for Adam.")
    parser.add_argument("--weight_decay", default=0.0, type=float, help="Weight decay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
    parser.add_argument(
        "--num_train_epochs", default=1.0, type=float, help="Total number of training epochs to perform."
    )
    parser.add_argument(
        "--max_steps",
        default=-1,
        type=int,
        help="If > 0: set total number of training steps to perform. Override num_train_epochs.",
    )
    parser.add_argument("--warmup_steps", default=0, type=int, help="Linear warmup over warmup_steps.")

    parser.add_argument("--logging_steps", type=int, default=500, help="Log every X updates steps.")
    parser.add_argument("--save_steps", type=int, default=500, help="Save checkpoint every X updates steps.")
    parser.add_argument(
        "--save_total_limit",
        type=int,
        default=None,
        help="Limit the total amount of checkpoints, delete the older checkpoints in the output_dir, does not delete by default",
    )
    parser.add_argument(
        "--eval_all_checkpoints",
        action="store_true",
        help="Evaluate all checkpoints starting with the same prefix as model_name_or_path ending and ending with step number",
    )
    parser.add_argument("--no_cuda", action="store_true", help="Avoid using CUDA when available")
    parser.add_argument(
        "--overwrite_output_dir", action="store_true", help="Overwrite the content of the output directory"
    )
    parser.add_argument(
        "--overwrite_cache", action="store_true", help="Overwrite the cached training and evaluation sets"
    )
    parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")

    parser.add_argument(
        "--fp16",
        action="store_true",
        help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit",
    )
    parser.add_argument(
        "--fp16_opt_level",
        type=str,
        default="O1",
        help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
        "See details at https://nvidia.github.io/apex/amp.html",
    )
    parser.add_argument("--local_rank", type=int, default=-1, help="For distributed training: local_rank")
    parser.add_argument("--server_ip", type=str, default="", help="For distant debugging.")
    parser.add_argument("--server_port", type=str, default="", help="For distant debugging.")
    args = parser.parse_args()

    if args.model_type in ["bert", "roberta", "distilbert", "camembert"] and not args.mlm:
        raise ValueError(
            "BERT and RoBERTa-like models do not have LM heads but masked LM heads. They must be run using the --mlm "
            "flag (masked language modeling)."
        )
    if args.eval_data_file is None and args.do_eval:
        raise ValueError(
            "Cannot do evaluation without an evaluation data file. Either supply a file to --eval_data_file "
            "or remove the --do_eval argument."
        )
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup distant debugging if needed
    if args.server_ip and args.server_port:
        # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
        import ptvsd

        print("Waiting for debugger attach")
        ptvsd.enable_attach(address=(args.server_ip, args.server_port), redirect_output=True)
        ptvsd.wait_for_attach()

    # Setup CUDA, GPU & distributed training
    if args.local_rank == -1 or args.no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        args.n_gpu = 0 if args.no_cuda else torch.cuda.device_count()
    else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.cuda.set_device(args.local_rank)
        device = torch.device("cuda", args.local_rank)
        torch.distributed.init_process_group(backend="nccl")
        args.n_gpu = 1
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    # Load pretrained model and tokenizer
    if args.local_rank not in [-1, 0]:
        torch.distributed.barrier()  # Barrier to make sure only the first process in distributed training download model & vocab

    if args.config_name:
        config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    elif args.model_name_or_path:
        config = AutoConfig.from_pretrained(args.model_name_or_path, cache_dir=args.cache_dir)
    else:
        # When we release a pip version exposing CONFIG_MAPPING,
        # we can do `config = CONFIG_MAPPING[args.model_type]()`.
        raise ValueError(
            "You are instantiating a new config instance from scratch. This is not supported, but you can do it from another script, save it,"
            "and load it from here, using --config_name"
        )

    if args.tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    elif args.model_name_or_path:
        tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=args.cache_dir)
    else:
        raise ValueError(
            "You are instantiating a new tokenizer from scratch. This is not supported, but you can do it from another script, save it,"
            "and load it from here, using --tokenizer_name"
        )

    if args.block_size <= 0:
        # args.block_size = tokenizer.max_len
        args.block_size = 512
        # Our input block size will be the max possible for the model
    else:
        # args.block_size = min(args.block_size, tokenizer.max_len)
        args.block_size = min(args.block_size, 512)

    if args.model_name_or_path:
        model = AutoModelWithLMHead.from_pretrained(
            args.model_name_or_path,
            from_tf=bool(".ckpt" in args.model_name_or_path),
            config=config,
            cache_dir=args.cache_dir,
        )
    else:
        logger.info("Training new model from scratch")
        model = AutoModelWithLMHead.from_config(config)

    model.to(args.device)

    if args.local_rank == 0:
        torch.distributed.barrier()  # End of barrier to make sure only the first process in distributed training download model & vocab

    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        if args.local_rank not in [-1, 0]:
            torch.distributed.barrier()  # Barrier to make sure only the first process in distributed training process the dataset, and the others will use the cache

        train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False)

        if args.local_rank == 0:
            torch.distributed.barrier()

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
        # Create output directory if needed
        if args.local_rank in [-1, 0]:
            os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results


if __name__ == "__main__":
    main()

Writing run_lm.py


In [ ]:
cs = []
with open('/content/drive/MyDrive/Adaptive_pretrain/cs.txt', 'r') as f:
    for txt in f.readlines():
        cs.append(txt)

max_len = int(len(cs) * 0.8)
train_cs = cs[:max_len]
dev_cs = cs[max_len+1:]

with open('/content/drive/MyDrive/Adaptive_pretrain/train.txt', 'w') as f:
    for txt in train_cs:
        f.write(txt)

with open('/content/drive/MyDrive/Adaptive_pretrain/dev.txt', 'w') as f:
    for txt in dev_cs:
        f.write(txt)

In [ ]:
cs[3]

'Estimating 3D human pose from monocular images demands large amounts of 3D pose and in-the-wild 2D pose annotated datasets which are costly and require sophisticated systems to acquire. In this regard, we propose a metric learning based approach to jointly learn a rich embedding and 3D pose regression from the embedding using multi-view synchronised videos of human motions and very limited 3D pose annotations. The inclusion of metric learning to the baseline pose estimation framework improves the performance by 21% when 3D supervision is limited. In addition, we make use of a person-identity based adversarial loss as additional weak supervision to outperform state-of-the-art whilst using a much smaller network. Lastly, but importantly, we demonstrate the advantages of the learned embedding and establish view-invariant pose retrieval benchmarks on two popular, publicly available multi-view human pose datasets, Human 3.6M and MPI-INF-3DHP, to facilitate future research.\n'

In [ ]:
!mkdir /content/data/
!cp /content/drive/MyDrive/RoBERTa_base_data/combined_train.txt /content/data
!cp /content/drive/MyDrive/RoBERTa_base_data/combined_dev.txt /content/data
!mv /content/data/combined_train.txt /content/data/train.txt
!mv /content/data/combined_dev.txt /content/data/dev.txt

In [ ]:
# Model paths
MODEL_TYPE = "roberta" #@param ["roberta", "bert"]
MODEL_DIR = "/content/models/roberta" #@param {type: "string"}
OUTPUT_DIR = "/content/models/roberta/output" #@param {type: "string"}
TRAIN_PATH = "/content/data/train.txt" #@param {type: "string"}
EVAL_PATH = "/content/data/dev.txt" #@param {type: "string"}

In [ ]:
# Command line
cmd = """python run_lm.py \
    --output_dir {output_dir} \
    --model_type {model_type} \
    --mlm \
    --config_name {config_name} \
    --tokenizer_name {tokenizer_name} \
    {line_by_line} \
    {should_continue} \
    {model_name_or_path} \
    --train_data_file {train_path} \
    --eval_data_file {eval_path} \
    --do_train \
    {do_eval} \
    {evaluate_during_training} \
    --overwrite_output_dir \
    --block_size 512 \
    --max_step 25 \
    --warmup_steps 10 \
    --learning_rate 5e-5 \
    --per_gpu_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --weight_decay 0.01 \
    --adam_epsilon 1e-6 \
    --max_grad_norm 100.0 \
    --save_total_limit 10 \
    --save_steps 10 \
    --logging_steps 2 \
    --seed 42
"""

In [ ]:
# Arguments for training from scratch. I turn off evaluate_during_training,
#   line_by_line, should_continue, and model_name_or_path.
train_params = {
    "output_dir": OUTPUT_DIR,
    "model_type": MODEL_TYPE,
    "config_name": MODEL_DIR,
    "tokenizer_name": MODEL_DIR,
    "train_path": TRAIN_PATH,
    "eval_path": EVAL_PATH,
    "do_eval": "--do_eval",
    "evaluate_during_training": "--evaluate_during_training",
    "line_by_line": "--line_by_line",
    "should_continue": "",
    "model_name_or_path": "",
}

In [ ]:
!{cmd.format(**train_params)}

08/30/2021 10:12:23 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/30/2021 10:12:23 - INFO - __main__ -   Training new model from scratch
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:892: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
08/30/2021 10:12:33 - INFO - __main__ -   Training/evaluation parameters Namespace(adam_epsilon=1e-06, block_size=512, cache_dir=None, config_name='/content/models/roberta', device=device(type='cuda'), do_eval=True, do_train=True, eval_all_checkpoints=False, eval_data_file='/content/data/dev.txt', evaluate_during_training=False, fp16=False, fp16_opt_level='O1', gradient_accumulation_steps=4, learning_rate=5e-0

In [ ]:
# !mkdir '/content/drive/MyDrive/Adaptive_pretrain/roberta_models'
!cp -R /content/models /content/drive/MyDrive/Adaptive_pretrain/roberta_models/
!cp -R /content/runs /content/drive/MyDrive/Adaptive_pretrain/roberta_models/

cp: cannot stat '/content/runs': No such file or directory


In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model='/content/models/roberta/output',
    tokenizer='/content/models/roberta/output'
)

In [ ]:
fill_mask("Natural language processing (NLP) or computational linguistics is one of the most <mask> technologies of the information age.")

[{'score': 0.0019420207245275378,
  'sequence': 'Natural language processing (NLP) or computational linguistics is one of the most. technologies of the information age.',
  'token': 18,
  'token_str': '.'},
 {'score': 0.0014330908888950944,
  'sequence': 'Natural language processing (NLP) or computational linguistics is one of the most, technologies of the information age.',
  'token': 16,
  'token_str': ','},
 {'score': 0.0013845879584550858,
  'sequence': 'Natural language processing (NLP) or computational linguistics is one of the most the technologies of the information age.',
  'token': 267,
  'token_str': ' the'},
 {'score': 0.0009771912591531873,
  'sequence': 'Natural language processing (NLP) or computational linguistics is one of the most of technologies of the information age.',
  'token': 292,
  'token_str': ' of'},
 {'score': 0.0002799422072712332,
  'sequence': 'Natural language processing (NLP) or computational linguistics is one of the most those technologies of the inf

In [ ]:
!cp /content/drive/MyDrive/RoBERTa_base_data/wiki.txt /content/data

In [ ]:
corpus = []
with open('/content/drive/MyDrive/RoBERTa_base_data/combined_corpus.txt', 'r') as f:
    for txt in f.readlines():
        corpus.append(txt)

max_len = int(len(corpus) * 0.8)
train_corpus = corpus[:max_len]
dev_corpus = corpus[max_len+1:]

with open('/content/data/train.txt', 'w') as f:
    for txt in train_corpus:
        f.write(txt)

with open('/content/data/dev.txt', 'w') as f:
    for txt in dev_corpus:
        f.write(txt)

In [ ]:
!sed -i '/^ *$/d' /content/drive/MyDrive/Adaptive_pretrain/.txt

In [ ]:
!pip install datasets

     |████████████████████████████████| 264 kB 7.3 MB/s 
     |████████████████████████████████| 118 kB 78.1 MB/s 
     |████████████████████████████████| 243 kB 70.1 MB/s 


In [ ]:
!cat /content/drive/MyDrive/RoBERTa_base_data/cc_news.txt /content/drive/MyDrive/RoBERTa_base_data/openweb.txt /content/drive/MyDrive/RoBERTa_base_data/all_books.txt /content/drive/MyDrive/RoBERTa_base_data/wiki.txt > /content/drive/MyDrive/RoBERTa_base_data/combined_corpus.txt

In [ ]:
!wc -l /content/drive/MyDrive/RoBERTa_base_data/combined_sents.txt

50172275 /content/drive/MyDrive/RoBERTa_base_data/combined_sents.txt


In [ ]:
max_len = int(50172275 * 0.8)

In [ ]:
max_len

40137820

In [ ]:
50172275 - max_len

10034455

In [ ]:
TRAIN_SIZE =  40137820#@param {type:"integer"}
VAL_SIZE =  10034455#@param {type: "integer"}

In [ ]:
!(head -n $TRAIN_SIZE /content/drive/MyDrive/RoBERTa_base_data/combined_sents.txt) > /content/drive/MyDrive/RoBERTa_base_data/combined_train.txt
!(sed -n {TRAIN_SIZE + 1},{TRAIN_SIZE + VAL_SIZE}p /content/drive/MyDrive/RoBERTa_base_data/combined_sents.txt) > /content/drive/MyDrive/RoBERTa_base_data/combined_dev.txt

In [ ]:
!wc -l /content/drive/MyDrive/RoBERTa_base_data/combined_train.txt

2620530 /content/drive/MyDrive/RoBERTa_base_data/combined_train.txt


In [ ]:
!wc -l /content/drive/MyDrive/RoBERTa_base_data/combined_dev.txt

655133 /content/drive/MyDrive/RoBERTa_base_data/combined_dev.txt


In [ ]:
!rm -r /content/models

In [ ]:
!rm -r /content/runs

In [ ]:
!mkdir /content/drive/MyDrive/RoBERTa_base_data/test_corpus

In [ ]:
!cp /content/drive/MyDrive/RoBERTa_base_data/combined_corpus.txt /content/drive/MyDrive/RoBERTa_base_data/test_corpus

In [ ]:
!split -n 10000 /content/drive/MyDrive/RoBERTa_base_data/test_corpus/combined_corpus.txt

In [ ]:
!cat xbmf

annister. So, which Premier League team has a deep seated hatred of Chelsea but are not contenders for the league crown? A good answer is Fulham. The West London club are traditional rivals of Chelsea and are not contenders for the Premier League crown.
2 House Tyrell - Manchester City
Over the first couple of season's of the Game of Thrones series, House Tyrell of Highgarden were on the periphery of ultimate power of Westeros. After the Battle of the Blackwater, the Tyrell's emerged as one of the dominant houses in Westeros. The most distinguishing characteristic of the Tyrell's is their wealth. They are responsible for feeding most of King's Landing. Another characteristic is their burgeoning rivalry with the Lannisters. The one Premier League team that closely resembles these characteristics is Manchester City. Manchester City were not a dominant team in the Premier League until very recently. In the last few years, however, they have emerged as a power in the English game. Their gr

In [ ]:
!cat /content/drive/MyDrive/Adaptive_pretrain/bio.txt /content/drive/MyDrive/Adaptive_pretrain/med.txt > /content/drive/MyDrive/Adaptive_pretrain/biomed.txt

In [ ]:
!sed -i '/^ *$/d' /content/drive/MyDrive/Adaptive_pretrain/biomed.txt

In [ ]:
!cp /content/drive/MyDrive/Adaptive_pretrain/cs.txt /content/drive/MyDrive/Adaptive_pretrain/cs_0830.txt

In [ ]:
!sed -i '/^ *$/d' /content/drive/MyDrive/Adaptive_pretrain/cs_0830.txt

In [ ]:
!wc -l /content/drive/MyDrive/Adaptive_pretrain/cs_0830.txt

106646 /content/drive/MyDrive/Adaptive_pretrain/cs_0830.txt


In [ ]:
max_len = 106646
train_len = int(max_len * 0.8)

In [ ]:
train_len


85316

In [ ]:
max_len - train_len


21330

In [ ]:
del max_len
del train_len
del fill_mask

import gc
gc.collect()

454

In [ ]:
## Fine tune
## CS

# Model paths
MODEL_TYPE = "roberta" #@param ["roberta", "bert"]
MODEL_DIR = "/content/models/roberta" #@param {type: "string"}
OUTPUT_DIR = "/content/models/roberta/output" #@param {type: "string"}
TRAIN_PATH = "/content/data/train01.txt" #@param {type: "string"}
EVAL_PATH = "/content/data/dev01.txt" #@param {type: "string"}

# TRAIN_SIZE = 85316 #@param {type:"integer"}
# VAL_SIZE = 21330 #@param {type: "integer"}

In [ ]:
!(head -n $TRAIN_SIZE /content/drive/MyDrive/Adaptive_pretrain/cs_0830.txt) > data/train.txt
!(sed -n {TRAIN_SIZE + 1},{TRAIN_SIZE + VAL_SIZE}p /content/drive/MyDrive/Adaptive_pretrain/cs_0830.txt) > data/dev.txt

In [ ]:
train_params = {
    "output_dir": OUTPUT_DIR,
    "model_type": MODEL_TYPE,
    "config_name": MODEL_DIR,
    "tokenizer_name": MODEL_DIR,
    "train_path": TRAIN_PATH,
    "eval_path": EVAL_PATH,
    "do_eval": "--do_eval",
    "evaluate_during_training": "",
    "line_by_line": "--line_by_line",
    "should_continue": "",
    "model_name_or_path": "",
}

In [ ]:
    !{cmd.format(**train_params)}

09/03/2021 01:29:49 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
09/03/2021 01:29:49 - INFO - __main__ -   Training new model from scratch
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:582: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
09/03/2021 01:29:59 - INFO - __main__ -   Training/evaluation parameters Namespace(adam_epsilon=1e-06, block_size=512, cache_dir=None, config_name='/content/models/roberta', device=device(type='cuda'), do_eval=True, do_train=True, eval_all_checkpoints=False, eval_data_file='/content/data/dev01.txt', evaluate_during_training=False, fp16=False, fp16_opt_level='O1', gradient_accumulation_steps=4, learning_rate=5e

In [ ]:
## DAPT
## 1. News data

!cp /content/articles1.csv.zip /content/drive/MyDrive/RoBERTa_domain_data/

In [ ]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz

--2021-09-03 02:09:40--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68275834 (65M) [application/x-gzip]
Saving to: ‘reviews_Sports_and_Outdoors_5.json.gz’

reviews_Sports_and_ 100%[===================>]  65.11M  3.30MB/s    in 9.8s    

2021-09-03 02:09:50 (6.61 MB/s) - ‘reviews_Sports_and_Outdoors_5.json.gz’ saved [68275834/68275834]



In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Movies_and_TV_5.json.gz

--2021-09-03 02:12:38--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Movies_and_TV_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791322468 (755M) [application/octet-stream]
Saving to: ‘Movies_and_TV_5.json.gz’

Movies_and_TV_5.jso 100%[===================>] 754.66M  93.1MB/s    in 8.3s    

2021-09-03 02:12:46 (91.3 MB/s) - ‘Movies_and_TV_5.json.gz’ saved [791322468/791322468]



In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Pet_Supplies_5.json.gz

--2021-09-03 02:13:12--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Pet_Supplies_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306131006 (292M) [application/octet-stream]
Saving to: ‘Pet_Supplies_5.json.gz’

Pet_Supplies_5.json 100%[===================>] 291.95M  90.4MB/s    in 3.4s    

2021-09-03 02:13:16 (85.4 MB/s) - ‘Pet_Supplies_5.json.gz’ saved [306131006/306131006]



In [ ]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Home_and_Kitchen_5.json.gz

--2021-09-03 02:13:33--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Home_and_Kitchen_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138126598 (132M) [application/x-gzip]
Saving to: ‘reviews_Home_and_Kitchen_5.json.gz’

reviews_Home_and_Ki 100%[===================>] 131.73M  2.01MB/s    in 61s     

2021-09-03 02:14:33 (2.17 MB/s) - ‘reviews_Home_and_Kitchen_5.json.gz’ saved [138126598/138126598]



In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Cell_Phones_and_Accessories_5.json.gz

--2021-09-03 02:16:14--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Cell_Phones_and_Accessories_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169071325 (161M) [application/octet-stream]
Saving to: ‘Cell_Phones_and_Accessories_5.json.gz’

Cell_Phones_and_Acc 100%[===================>] 161.24M  86.6MB/s    in 1.9s    

2021-09-03 02:16:16 (86.6 MB/s) - ‘Cell_Phones_and_Accessories_5.json.gz’ saved [169071325/169071325]



In [ ]:
import os

os.listdir('/content/')

['.config',
 'Pet_Supplies_5.json.gz',
 'articles1.csv.zip',
 'Movies_and_TV_5.json.gz',
 'reviews_Home_and_Kitchen_5.json.gz',
 'Cell_Phones_and_Accessories_5.json.gz',
 'drive',
 'reviews_Sports_and_Outdoors_5.json.gz',
 'sample_data']

In [ ]:
jsongzs = [x for x in os.listdir('/content/') if x[-8:] == '.json.gz']

In [ ]:
jsongzs

['Pet_Supplies_5.json.gz',
 'Movies_and_TV_5.json.gz',
 'reviews_Home_and_Kitchen_5.json.gz',
 'Cell_Phones_and_Accessories_5.json.gz',
 'reviews_Sports_and_Outdoors_5.json.gz']

In [ ]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF(jsongzs[0])

In [ ]:
len(df['reviewText'].values)

296337

In [ ]:
review = []
for i in range(len(jsongzs)):
    df = getDF(jsongzs[i])
    review.append(df['reviewText'].values)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
len(review[0])

2098325

In [ ]:
len(review[0][0])

389

In [ ]:
review[1][0]

"So sorry I didn't purchase this years ago when it first came out!!  This is very good and entertaining!  We absolutely loved it and anticipate seeing it repeatedly.  We actually wore out the cassette years back, so we also purchased this same product on cd.  Best purchase we made out of all!  Would purchase on dvd if we could find one."

In [ ]:
review[0]

array(["I purchased this cd for my Pocket Parrot. It has both a woman and a man speaking. Pet store recommends to buy a female voice Only on bird cds because birds respond to female voice better. Con: there's some real stupid things said on this CD. Prefer female voice because pet shop recommends female voice. Pros: it's cheap. I will review again in 6 months see if my Polly picks up few words.",
       "Maybe it's just my Amazon parrot, but she's not picking up very quickly or very well.",
       'I bought this to help me teach my blue quaker named Booger to talk and it sure has helped. I put it in the CD player and let it play all day while I am at work. It might drive Booger nuts, but he sure has learned a lot of words.',
       ...,
       'My destroyer French Bulldog was not able to destroy the squeaker within the usual 2 minutes! I was surprised and happy. Love the toy and would recommend it.',
       "This is one of my dog's favorite toys, but all three quickly lost their squeak

In [ ]:
import numpy as np
flat_review = np.array(review, dtype=object).flatten()

In [ ]:
len(flat_review[3])

1128437

In [ ]:
len(flat_review[0][0])

389

In [ ]:
flat_review[0][0]

"I purchased this cd for my Pocket Parrot. It has both a woman and a man speaking. Pet store recommends to buy a female voice Only on bird cds because birds respond to female voice better. Con: there's some real stupid things said on this CD. Prefer female voice because pet shop recommends female voice. Pros: it's cheap. I will review again in 6 months see if my Polly picks up few words."

In [ ]:
sent_tokenize(flat_review[0][0])

['I purchased this cd for my Pocket Parrot.',
 'It has both a woman and a man speaking.',
 'Pet store recommends to buy a female voice Only on bird cds because birds respond to female voice better.',
 "Con: there's some real stupid things said on this CD.",
 'Prefer female voice because pet shop recommends female voice.',
 "Pros: it's cheap.",
 'I will review again in 6 months see if my Polly picks up few words.']

In [ ]:
from nltk.tokenize import sent_tokenize

sent_review = []
for i in range(len(flat_review)):
    for j in range(len(flat_review[i])):
        try:
            sent_review.extend(sent_tokenize(flat_review[i][j]))
        except:
            pass

print(len(sent_review))

33997335


In [ ]:
with open('/content/drive/MyDrive/RoBERTa_domain_data/review.txt', 'w') as f:
    for txt in sent_review:
        f.write("%s\n" % txt)

In [ ]:
!wc -w /content/drive/MyDrive/RoBERTa_domain_data/review.txt

515381387 /content/drive/MyDrive/RoBERTa_domain_data/review.txt


In [ ]:
!mv /content/Cell_Phones_and_Accessories_5.json.gz /content/drive/MyDrive/RoBERTa_domain_data/
!mv /content/Movies_and_TV_5.json.gz /content/drive/MyDrive/RoBERTa_domain_data/
!mv /content/Pet_Supplies_5.json.gz /content/drive/MyDrive/RoBERTa_domain_data/
!mv /content/reviews_Home_and_Kitchen_5.json.gz /content/drive/MyDrive/RoBERTa_domain_data/
!mv /content/reviews_Sports_and_Outdoors_5.json.gz /content/drive/MyDrive/RoBERTa_domain_data/

In [ ]:
!unzip /content/articles1.csv.zip

Archive:  /content/articles1.csv.zip
  inflating: articles1.csv           


In [ ]:
import pandas as pd

df = pd.read_csv('/content/articles1.csv')

In [ ]:
df.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [ ]:
contents = df['content'].values

news = []
for txt in contents:
    news.extend(sent_tokenize(txt))

In [ ]:
len(contents[0].split(" "))

920

In [ ]:
len(news)

1520613

In [ ]:
!unzip /content/articles2.csv.zip
!unzip /content/articles3.csv.zip

df2 = pd.read_csv('/content/articles2.csv')
df3 = pd.read_csv('/content/articles3.csv')

Archive:  /content/articles2.csv.zip
  inflating: articles2.csv           
Archive:  /content/articles3.csv.zip
  inflating: articles3.csv           


In [ ]:
contents = df['content'].values

for txt in contents:
    news.extend(sent_tokenize(txt))

In [ ]:
contents = df['content'].values

for txt in contents:
    news.extend(sent_tokenize(txt))

In [ ]:
len(news)

4561839

In [ ]:
news[0]

'WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win.'

In [ ]:
with open('/content/drive/MyDrive/RoBERTa_domain_data/domain_news.txt', 'w') as f:
    for txt in news:
        f.write("%s\n" % txt)

In [ ]:
!wc -w /content/drive/MyDrive/RoBERTa_domain_data/domain_news.txt

96913761 /content/drive/MyDrive/RoBERTa_domain_data/domain_news.txt


In [ ]:
!wc -w /content/drive/MyDrive/Adaptive_pretrain/cs.txt

16405408 /content/drive/MyDrive/Adaptive_pretrain/cs.txt


In [ ]:
cs = []
with open('/content/drive/MyDrive/Adaptive_pretrain/cs.txt', 'r') as f:
    for txt in f.readlines():
        cs.extend(sent_tokenize(txt))
biomed = []
with open('/content/drive/MyDrive/Adaptive_pretrain/biomed.txt', 'r') as f:
    for txt in f.readlines():
        biomed.extend(sent_tokenize(txt))

with open('/content/drive/MyDrive/RoBERTa_domain_data/domain_cs.txt', 'w') as f:
    for txt in cs:
        f.write("%s\n" % txt)

with open('/content/drive/MyDrive/RoBERTa_domain_data/domain_biomed.txt', 'w') as f:
    for txt in biomed:
        f.write("%s\n" % txt)

In [ ]:
!wc -l /content/drive/MyDrive/RoBERTa_domain_data/domain_biomed.txt
!wc -l /content/drive/MyDrive/RoBERTa_domain_data/domain_cs.txt
!wc -l /content/drive/MyDrive/RoBERTa_domain_data/domain_review.txt
!wc -l /content/drive/MyDrive/RoBERTa_domain_data/domain_news.txt

905793 /content/drive/MyDrive/RoBERTa_domain_data/domain_biomed.txt
712923 /content/drive/MyDrive/RoBERTa_domain_data/domain_cs.txt
35265731 /content/drive/MyDrive/RoBERTa_domain_data/domain_review.txt
4561839 /content/drive/MyDrive/RoBERTa_domain_data/domain_news.txt


In [ ]:
712923 * 0.8

570338.4

In [ ]:
712923 - 570338

142585

In [ ]:
!mkdir -p /content/data/biomed
!mkdir -p /content/data/cs
!mkdir -p /content/data/review
!mkdir -p /content/data/news

TRAIN_SIZE =  570338#@param {type:"integer"}
VAL_SIZE =  142585#@param {type: "integer"}

In [ ]:
!(head -n $TRAIN_SIZE /content/drive/MyDrive/RoBERTa_domain_data/domain_cs.txt) > /content/data/cs/train.txt
!(sed -n {TRAIN_SIZE + 1},{TRAIN_SIZE + VAL_SIZE}p /content/drive/MyDrive/RoBERTa_domain_data/domain_cs.txt) > /content/data/cs/dev.txt

!(head -n $TRAIN_SIZE /content/drive/MyDrive/RoBERTa_domain_data/domain_biomed.txt) > /content/data/biomed/train.txt
!(sed -n {TRAIN_SIZE + 1},{TRAIN_SIZE + VAL_SIZE}p /content/drive/MyDrive/RoBERTa_domain_data/domain_biomed.txt) > /content/data/biomed/dev.txt

!(head -n $TRAIN_SIZE /content/drive/MyDrive/RoBERTa_domain_data/domain_review.txt) > /content/data/review/train.txt
!(sed -n {TRAIN_SIZE + 1},{TRAIN_SIZE + VAL_SIZE}p /content/drive/MyDrive/RoBERTa_domain_data/domain_review.txt) > /content/data/review/dev.txt

!(head -n $TRAIN_SIZE /content/drive/MyDrive/RoBERTa_domain_data/domain_news.txt) > /content/data/news/train.txt
!(sed -n {TRAIN_SIZE + 1},{TRAIN_SIZE + VAL_SIZE}p /content/drive/MyDrive/RoBERTa_domain_data/domain_news.txt) > /content/data/news/dev.txt

In [ ]:
!rm -r /content/models

In [ ]:
import json

config = {
    "architectures": [
		"RobertaForMaskedLM"
	],
	"attention_probs_dropout_prob": 0.1,
	"hidden_act": "gelu",
	"hidden_dropout_prob": 0.1,
	"hidden_size": 768,
	"initializer_range": 0.02,
	"intermediate_size": 3072,
	"layer_norm_eps": 1e-05,
	"max_position_embeddings": 514,
	"model_type": "roberta",
	"num_attention_heads": 12,
	"num_hidden_layers": 12,
	"type_vocab_size": 1,
	"vocab_size": 50265
}

# with open("/content/models/roberta/config.json", 'w') as fp:
#     json.dump(config, fp)

tokenizer_config = {"max_len": 512}

# with open("/content/models/roberta/tokenizer_config.json", 'w') as fp:
#     json.dump(tokenizer_config, fp)

In [ ]:
# Command line
cmd = """python run_lm.py \
    --output_dir {output_dir} \
    --model_type {model_type} \
    --mlm \
    --config_name {config_name} \
    --tokenizer_name {tokenizer_name} \
    {line_by_line} \
    {should_continue} \
    {model_name_or_path} \
    --train_data_file {train_path} \
    --eval_data_file {eval_path} \
    --num_train_epochs {num_train_epochs} \
    --do_train \
    {do_eval} \
    {evaluate_during_training} \
    --overwrite_output_dir \
    --block_size 512 \
    --max_step 25 \
    --warmup_steps 10 \
    --learning_rate 5e-5 \
    --per_gpu_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --weight_decay 0.01 \
    --adam_epsilon 1e-6 \
    --max_grad_norm 100.0 \
    --save_total_limit 10 \
    --save_steps 10 \
    --logging_steps 2 \
    --seed 42 \
    |& tee -a /content/drive/MyDrive/Adaptive_pretrain/domain_output.txt
"""

In [ ]:
# Train RoBERTa domain

import os
# from distutils.dir_util import copy_tree
domains = ['cs', 'biomed', 'review', 'news']
base = '/content/data/'
for d in domains:
    print("__________START " + f"{d}__________")
    train_file = f"{d}/train.txt"
    dev_file = f"{d}/dev.txt"
    # COPY model
    !cp -R /content/drive/MyDrive/Adaptive_pretrain/sentenced/roberta_models_base9/models/ /content/
    !cp -R /content/drive/MyDrive/Adaptive_pretrain/sentenced/roberta_models_base9/runs/ /content/
    !cp /content/drive/MyDrive/Adaptive_pretrain/tokenizer.json /content/models/roberta/

    with open("/content/models/roberta/config.json", 'w') as fp:
        json.dump(config, fp)

    with open("/content/models/roberta/tokenizer_config.json", 'w') as fp:
        json.dump(tokenizer_config, fp)
    
    # Model paths
    MODEL_TYPE = "roberta"
    MODEL_DIR = "/content/models/roberta"
    OUTPUT_DIR = "/content/models/roberta/output_base"
    TRAIN_PATH = "/content/data/"+train_file
    EVAL_PATH = "/content/data/"+dev_file

    train_params = {
        "output_dir": OUTPUT_DIR,
        "model_type": MODEL_TYPE,
        "config_name": MODEL_DIR,
        "tokenizer_name": MODEL_DIR,
        "train_path": TRAIN_PATH,
        "eval_path": EVAL_PATH,
        "num_train_epochs": 1.0,
        "do_eval": "--do_eval",
        "evaluate_during_training": "",
        "line_by_line": "--line_by_line",
        "should_continue": "--should_continue",
        "model_name_or_path": "",
    }

    # if num_corpus != 0:
    #     train_params['should_continue'] = "--should_continue"
    
    !{cmd.format(**train_params)}

    save_dir = f'/content/drive/MyDrive/Adaptive_pretrain/roberta_models_domain_{d}/'
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    !cp -R /content/models "$save_dir"
    !cp -R /content/runs "$save_dir"

__________START cs__________
09/03/2021 05:44:51 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:592: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
09/03/2021 05:44:59 - INFO - __main__ -   Training/evaluation parameters Namespace(adam_epsilon=1e-06, block_size=512, cache_dir=None, config_name='/content/models/roberta', device=device(type='cuda'), do_eval=True, do_train=True, eval_all_checkpoints=False, eval_data_file='/content/data/cs/dev.txt', evaluate_during_training=False, fp16=False, fp16_opt_level='O1', gradient_accumulation_steps=4, learning_rate=5e-05, line_by_line=True, local_rank=-1, loggi